In [1]:
## IT'S DANGEROUS TO GO ALONE! TAKE THIS:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#comebacktothis wherever I could improve or left something ugly
#gethelp wherever I need it

*In order to give my computer's RAM a break, I broke the wrangling/cleaning up into three notebooks.
This is part two of the miniseries - getting the data from the United States Department of Agriculture's National Agricultural Statistics Service cleaned and into one single dataframe for the 20 years of data we will be working on.*

*The 'rough' csv's ('') have been zipped and transferred to the coding storage folder on my D drive, in the Capstone subfolder:*
* D:\gdsak\Coding_Storage\Capstone\roughUSDA

# USDA Census and Survey of Agriculture

## Data Sources

We have three data sources, each with slightly different types of data. The goal is to have either weekly (with imputed/interpolated weather data) or monthly (with binned and averaged drought data)

- NOAA Weather data

- US Drought monitor

- **USDA/NASS Census and Survey of Ag data** - We'll be working on this in this notebook
    * This is 8 spreadsheets (10, if we are including the organic data in the formal modeling, but probably not.) with both county and state level data for 4 types of data: Fertilizer costs, chemical costs, number of operations, number of acres. Some of the 'crosses' (acres per operation, fertilizer per operation etc) are calculated. All of these need to be vertically translated into variable columns and combined into one spreadsheet. We have full data for 15 states and partial data for all 50.
    * This data is sourced from The National Agricultural Statistics Service 'Quick Stats' self-service searchable database:
    https://quickstats.nass.usda.gov/
    
The Census of Agriculture has a glossary that we will reference as we get rid of missing values.

# comebacktothis
circle back to the acquisition notebook and walk through the spreadsheets you generated from the USDA/NASS quickstats service. The explanation that you needed to use the quickstats application to make sure you were getting the data you actually wanted is totally valid. If the API is harder to use than the self serve, you're gonna use the self serve...

In [2]:
#reading in the census of agriculture 
coa = pd.read_csv('data/COA_ChemFertOps&Acres_Counties.csv', thousands=',')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
coa

C:\Users\gdsak\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,County ANSI,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,CENSUS,2017,YEAR,NaN,COUNTY,ARKANSAS,5,CENTRAL,50,CONWAY,29,NaN,NaN,0,NaN,CHEMICAL TOTALS,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $",TOTAL,NOT SPECIFIED,"1,395,000",19.1
1,CENSUS,2017,YEAR,NaN,COUNTY,ARKANSAS,5,CENTRAL,50,CONWAY,29,NaN,NaN,0,NaN,CHEMICAL TOTALS,CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,TOTAL,NOT SPECIFIED,375,23.2
2,CENSUS,2017,YEAR,NaN,COUNTY,ARKANSAS,5,CENTRAL,50,CONWAY,29,NaN,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - ACRES OPERATED,TOTAL,NOT SPECIFIED,"171,893",15.7
3,CENSUS,2017,YEAR,NaN,COUNTY,ARKANSAS,5,CENTRAL,50,CONWAY,29,NaN,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TOTAL,NOT SPECIFIED,768,20.0
4,CENSUS,2017,YEAR,NaN,COUNTY,ARKANSAS,5,CENTRAL,50,CONWAY,29,NaN,NaN,0,NaN,FERTILIZER TOTALS,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $",TOTAL,NOT SPECIFIED,"2,350,000",20.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36345,CENSUS,2002,YEAR,NaN,COUNTY,WISCONSIN,55,WEST CENTRAL,40,TREMPEALEAU,121,NaN,NaN,0,NaN,CHEMICAL TOTALS,CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,TOTAL,NOT SPECIFIED,671,NaN
36346,CENSUS,2002,YEAR,NaN,COUNTY,WISCONSIN,55,WEST CENTRAL,40,TREMPEALEAU,121,NaN,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - ACRES OPERATED,TOTAL,NOT SPECIFIED,"367,830",NaN
36347,CENSUS,2002,YEAR,NaN,COUNTY,WISCONSIN,55,WEST CENTRAL,40,TREMPEALEAU,121,NaN,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TOTAL,NOT SPECIFIED,"1,744",NaN
36348,CENSUS,2002,YEAR,NaN,COUNTY,WISCONSIN,55,WEST CENTRAL,40,TREMPEALEAU,121,NaN,NaN,0,NaN,FERTILIZER TOTALS,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $",TOTAL,NOT SPECIFIED,"4,328,000",NaN


In [3]:
coa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36350 entries, 0 to 36349
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Program           36350 non-null  object 
 1   Year              36350 non-null  int64  
 2   Period            36350 non-null  object 
 3   Week Ending       0 non-null      float64
 4   Geo Level         36350 non-null  object 
 5   State             36350 non-null  object 
 6   State ANSI        36350 non-null  int64  
 7   Ag District       36350 non-null  object 
 8   Ag District Code  36350 non-null  int64  
 9   County            36350 non-null  object 
 10  County ANSI       36350 non-null  int64  
 11  Zip Code          0 non-null      float64
 12  Region            0 non-null      float64
 13  watershed_code    36350 non-null  int64  
 14  Watershed         0 non-null      float64
 15  Commodity         36350 non-null  object 
 16  Data Item         36350 non-null  object

The first thing to note is that there's plenty of 'junk' columns that we can remove by reimporting the dataframe without them. 

The **Program** column is temporarily useful as this will differentiate where our data is coming from, the Census of Agriculture or the Survey of Agriculture. After we combine this dataframe with the Survey of Agriculture data we will likely drop it.

**Year, State, and County** are all worth keeping for now, though we may need to change the case of the string data in **State** and **County** when we merge the data with our  **drought** dataframe.

**State ANSI and County ANSI** *should* be equivolent to the FIPS numbers we have seen previously. We'll double check this and if they match the expected FIPS number then we can add in the leading zeros that we need for each and combine them into one FIPS column

**Data Item and Value** columns hold our variable data--they will need to be transformed and separated into 6 columns that we will rename as follows: 

**CHEMICAL TOTALS - EXPENSE, MEASURED IN $** - chem_expense_dollars

**CHEMICAL TOTALS - OPERATIONS WITH EXPENSE** - chem_operations

**FARM OPERATIONS - ACRES OPERATED** - acres_operated

**FARM OPERATIONS - NUMBER OF OPERATIONS** - operations

**FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $** - fert_expense_dollars

**FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - OPERATIONS WITH EXPENSE** - fert_operations

All the other columns are either filled with NaN values or are not meaningful for our purposes. We'll leave the remaining columns out of our second pass at the read_csv function, effectively dropping them. The columns we need to keep are:

 * 'Program', 'Year', 'State', 'State ANSI', 'County','County ANSI', 'Data Item', 'Value'

The other noteworthy piece about the data is that the number of rows seems a bit off. We have 4 years of data and 6 variables selected for each. If the counties are static year to year and we aren't missing rows, then our total rows should be evenly divisible by 24 (4 years x 6 variables). We can reimport the dataframe without the 'junk' columns and then take a look at the county values.

In [4]:
coa = pd.read_csv('data/COA_ChemFertOps&Acres_Counties.csv',
                  thousands=',',
                  header=0,
                  names=['Program', 'Year', 'Period','Week Ending', 'Geo Level', 'State', 'State ANSI', 'Ag District', 'Ag District Code', 'County','County ANSI', 'Zip Code', 'Region','watershed_code', 'Watershed', 'Commodity', 'Data Item', 'Domain', 'Domain Category', 'Value', 'CV (%)'],
                  usecols=['Program', 'Year', 'State', 'State ANSI', 'County','County ANSI', 'Data Item', 'Value'])
coa

,Program,Year,State,State ANSI,County,County ANSI,Data Item,Value
0,CENSUS,2017,ARKANSAS,5,CONWAY,29,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $","1,395,000"
1,CENSUS,2017,ARKANSAS,5,CONWAY,29,CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,375
2,CENSUS,2017,ARKANSAS,5,CONWAY,29,FARM OPERATIONS - ACRES OPERATED,"171,893"
3,CENSUS,2017,ARKANSAS,5,CONWAY,29,FARM OPERATIONS - NUMBER OF OPERATIONS,768
4,CENSUS,2017,ARKANSAS,5,CONWAY,29,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","2,350,000"
...,...,...,...,...,...,...,...,...
36345,CENSUS,2002,WISCONSIN,55,TREMPEALEAU,121,CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,671
36346,CENSUS,2002,WISCONSIN,55,TREMPEALEAU,121,FARM OPERATIONS - ACRES OPERATED,"367,830"
36347,CENSUS,2002,WISCONSIN,55,TREMPEALEAU,121,FARM OPERATIONS - NUMBER OF OPERATIONS,"1,744"
36348,CENSUS,2002,WISCONSIN,55,TREMPEALEAU,121,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","4,328,000"


In [5]:
print(f'...unfortunately, {coa.shape[0]} divided by 24 is {round((coa.shape[0]/24),2)}.\n\
So we may some missing data or changes year-to-year in the counties.')

...unfortunately, 36350 divided by 24 is 1514.58.
So we may some missing data or changes year-to-year in the counties.


In [6]:
#checking the number of counties for each state.
(coa['State'].value_counts()/24)

TEXAS             253.500000
GEORGIA           158.916667
MISSOURI          114.000000
KANSAS            105.000000
ILLINOIS          102.000000
NORTH CAROLINA    100.000000
IOWA               99.000000
NEBRASKA           93.000000
INDIANA            92.000000
MINNESOTA          86.916667
ARKANSAS           75.000000
WISCONSIN          71.583333
FLORIDA            67.000000
CALIFORNIA         57.666667
WASHINGTON         39.000000
Name: State, dtype: float64

Most of the states divide evenly by 24 while **Texas, Georgia, Minnesota, Wisconsin, and California** have remainders.
During our data acquisition we were able to scrape county data from Wikipedia--we can pull that in to verify the current number of counties for each of the 15 states we are examining.

In [7]:
county_info = pd.read_csv('data/roughFIPSandClimDiv_df.csv')
county_info

,CLIMDIV_ID,FIPS,Sort [1],State,County [2],County Seat(s) [3],Population(2010),Land Areakm²,Land Areami²,Water Areakm²,Water Areami²,Total Areakm²,Total Areami²,Latitude,Longitude
0,101.0,1033,17.0,AL,Colbert,Tuscumbia,54428.0,1534.877,592.619,76.431,29.510,1611.308,622.129,+34.703112°,–87.801457°
1,101.0,1059,30.0,AL,Franklin,Russellville,31704.0,1641.588,633.821,32.898,12.702,1674.486,646.523,+34.441988°,–87.842815°
2,101.0,1077,39.0,AL,Lauderdale,Florence,92709.0,1729.328,667.697,138.027,53.293,1867.355,720.990,+34.904122°,–87.650997°
3,101.0,1079,40.0,AL,Lawrence,Moulton,34339.0,1788.847,690.678,68.688,26.521,1857.535,717.199,+34.529776°,–87.321865°
4,101.0,1083,42.0,AL,Limestone,Athens,82782.0,1450.228,559.936,122.382,47.252,1572.610,607.188,+34.810239°,–86.981400°
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140,NaN,26083,1273.0,MI,Keweenaw,Eagle River,2156.0,1398.883,540.112,14053.476,5426.078,15452.359,5966.190,+47.681981°,–88.148802°
3141,NaN,46102,2418.0,SD,Oglala Lakota [11],— [11],13586.0,5423.170,2093.898,7.126,2.751,5430.296,2096.649,+43.341937°,–102.559480°
3142,NaN,48007,2527.0,TX,Aransas,Rockport,23158.0,652.869,252.074,714.621,275.917,1367.490,527.991,+28.104225°,–96.977983°
3143,NaN,51735,2943.0,VA,Poquoson [9],—,12150.0,39.670,15.317,163.452,63.109,203.122,78.426,+37.128360°,–76.303534°


In [8]:
#checking the number of counties for each state circa 2022
county_info['State'].value_counts()

TX    254
GA    159
VA    134
KY    120
MO    115
KS    105
IL    102
NC    100
IA     99
TN     95
NE     93
IN     92
OH     88
MN     87
MI     83
MS     82
OK     77
AR     75
WI     72
FL     67
PA     67
AL     67
SD     66
LA     64
CO     64
NY     62
CA     58
MT     56
WV     55
ND     53
SC     46
ID     44
WA     39
OR     36
NM     33
AK     29
UT     29
MD     24
WY     23
NJ     21
NV     17
ME     16
AZ     15
MA     14
VT     14
NH     10
CT      8
RI      5
HI      5
DE      3
DC      1
Name: State, dtype: int64

As of 2022:
* Texas (TX) has 254 counties
* Georgia (GA) has 159 counties
* Minnesota (MN) has 87 counties
* Wisconsin (WI) has 72 counties
* California (CA) has 58 counties

So some time in our 20 year span each of these states may have added a county (or changed county names). Once our dataframe is tidier we can keep these states in mind as we check for missing/null values.

# Test Drive #1:
### Getting the Census of Ag data into tidy columns.

Our variable values are currently (and unhelpfully) stacked on top of each other. In order to split these into columns, we can use a function similar to the **spread** function that is common to the language **R**. Referencing our friends Wickham and Grolemund in thier textbook "R for Data Science":

"Spreading is the opposite of gathering. You use it when an observation is scattered across multiple rows."

As I mentioned in part 1 of the clean up, if I can think of something I want to code, someone has already coded it. In this case, the same someone that I borrowed the **gather** function from.

We'll follow a similar workflow and create a small subset of the dataframe to test this out before unleashing it on 30,000+ rows of data.

In [9]:
#borrowed from:
# https://gist.github.com/derekpowell/5f97dabdd0730e68380fa1a00cd34ac4
#thanks again, Professor Powell! You probably won't ever see this.

def spread( df, index, columns, values ):
    return df.pivot(index, columns, values).reset_index(level=index).rename_axis(None,axis=1)

In [10]:
#making a short copy to test drive
coa_small = coa[:12].copy()
coa_small

,Program,Year,State,State ANSI,County,County ANSI,Data Item,Value
0,CENSUS,2017,ARKANSAS,5,CONWAY,29,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $","1,395,000"
1,CENSUS,2017,ARKANSAS,5,CONWAY,29,CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,375
2,CENSUS,2017,ARKANSAS,5,CONWAY,29,FARM OPERATIONS - ACRES OPERATED,"171,893"
3,CENSUS,2017,ARKANSAS,5,CONWAY,29,FARM OPERATIONS - NUMBER OF OPERATIONS,768
4,CENSUS,2017,ARKANSAS,5,CONWAY,29,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","2,350,000"
5,CENSUS,2017,ARKANSAS,5,CONWAY,29,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - OPERATIONS WITH EXPENSE",405
6,CENSUS,2017,ARKANSAS,5,FAULKNER,45,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $","921,000"
7,CENSUS,2017,ARKANSAS,5,FAULKNER,45,CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,388
8,CENSUS,2017,ARKANSAS,5,FAULKNER,45,FARM OPERATIONS - ACRES OPERATED,"200,539"
9,CENSUS,2017,ARKANSAS,5,FAULKNER,45,FARM OPERATIONS - NUMBER OF OPERATIONS,"1,191"


In [11]:
#Trying out the spread function
coa_small_spread = spread(df=coa_small,
       index='County ANSI',
       columns='Data Item',
       values='Value')
coa_small_spread

,County ANSI,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $",CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,FARM OPERATIONS - ACRES OPERATED,FARM OPERATIONS - NUMBER OF OPERATIONS,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - OPERATIONS WITH EXPENSE"
0,29,"1,395,000",375,"171,893",768,"2,350,000",405
1,45,"921,000",388,"200,539","1,191","1,559,000",419


In [12]:
coa_small.merge(coa_small_spread, on='County ANSI')

,Program,Year,State,State ANSI,County,County ANSI,Data Item,Value,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $",CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,FARM OPERATIONS - ACRES OPERATED,FARM OPERATIONS - NUMBER OF OPERATIONS,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - OPERATIONS WITH EXPENSE"
0,CENSUS,2017,ARKANSAS,5,CONWAY,29,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $","1,395,000","1,395,000",375,"171,893",768,"2,350,000",405
1,CENSUS,2017,ARKANSAS,5,CONWAY,29,CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,375,"1,395,000",375,"171,893",768,"2,350,000",405
2,CENSUS,2017,ARKANSAS,5,CONWAY,29,FARM OPERATIONS - ACRES OPERATED,"171,893","1,395,000",375,"171,893",768,"2,350,000",405
3,CENSUS,2017,ARKANSAS,5,CONWAY,29,FARM OPERATIONS - NUMBER OF OPERATIONS,768,"1,395,000",375,"171,893",768,"2,350,000",405
4,CENSUS,2017,ARKANSAS,5,CONWAY,29,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","2,350,000","1,395,000",375,"171,893",768,"2,350,000",405
5,CENSUS,2017,ARKANSAS,5,CONWAY,29,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - OPERATIONS WITH EXPENSE",405,"1,395,000",375,"171,893",768,"2,350,000",405
6,CENSUS,2017,ARKANSAS,5,FAULKNER,45,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $","921,000","921,000",388,"200,539","1,191","1,559,000",419
7,CENSUS,2017,ARKANSAS,5,FAULKNER,45,CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,388,"921,000",388,"200,539","1,191","1,559,000",419
8,CENSUS,2017,ARKANSAS,5,FAULKNER,45,FARM OPERATIONS - ACRES OPERATED,"200,539","921,000",388,"200,539","1,191","1,559,000",419
9,CENSUS,2017,ARKANSAS,5,FAULKNER,45,FARM OPERATIONS - NUMBER OF OPERATIONS,"1,191","921,000",388,"200,539","1,191","1,559,000",419


Great! We can use that to create separate columns for our variables. We'll naturally create quite a few duplicate values (we will only need 1 in 6 rows), but we should be drop the **Data Item** and **Values** columns and then drop duplicates. 

Let's set up a **FIPSYear** column by adding leading zeros to the State and County ANSI columns and combining them with the year column.

First, a quick double check that the ANSI numbers match the FIPS numbers. We can check (*scenic!*) Trempealeau county, Wisconsin. Its State ANSI (55) should be the first two digits of the FIPS number and its County ANSI (121) the last three, so **55121**. We'll use our county_info dataframe to check.

In [13]:
#checking that the ANSI number 55121 pulls up Trempealeau County, WI
#it really is a lovely area--the "driftless" region surrounding the upper mississippi river
county_info.loc[county_info['FIPS']==55121]

,CLIMDIV_ID,FIPS,Sort [1],State,County [2],County Seat(s) [3],Population(2010),Land Areakm²,Land Areami²,Water Areakm²,Water Areami²,Total Areakm²,Total Areami²,Latitude,Longitude
3042,4704.0,55121,3110.0,WI,Trempealeau,Whitehall,28816.0,1898.37,732.965,23.371,9.024,1921.741,741.989,+44.303050°,–91.358867°


Our ANSI and FIPS numbers appear to be interchangeable. A search for information on the ANSI numbers yielded more information from the US Census Bureau: https://www.census.gov/library/reference/code-lists/ansi.html#county

With these reassurances, we can confidently convert our State and County ANSI numbers to one **FIPS** column and use that **FIPS** column to make an index for our merge, **FIPSYear**.

In [14]:
#enough talk and due diligence, lets add some leading zeros already

#comebacktothis - should I make a function out of this orrrrr...?

#convert to strings
coa['State ANSI'] = coa['State ANSI'].astype(str)
coa['County ANSI'] = coa['County ANSI'].astype(str)

coa['State ANSI'] = coa['State ANSI'].str.zfill(2) # adding one leading zero to the state
coa['County ANSI'] = coa['County ANSI'].str.zfill(3)# and one or two to the county
coa

,Program,Year,State,State ANSI,County,County ANSI,Data Item,Value
0,CENSUS,2017,ARKANSAS,05,CONWAY,029,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $","1,395,000"
1,CENSUS,2017,ARKANSAS,05,CONWAY,029,CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,375
2,CENSUS,2017,ARKANSAS,05,CONWAY,029,FARM OPERATIONS - ACRES OPERATED,"171,893"
3,CENSUS,2017,ARKANSAS,05,CONWAY,029,FARM OPERATIONS - NUMBER OF OPERATIONS,768
4,CENSUS,2017,ARKANSAS,05,CONWAY,029,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","2,350,000"
...,...,...,...,...,...,...,...,...
36345,CENSUS,2002,WISCONSIN,55,TREMPEALEAU,121,CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,671
36346,CENSUS,2002,WISCONSIN,55,TREMPEALEAU,121,FARM OPERATIONS - ACRES OPERATED,"367,830"
36347,CENSUS,2002,WISCONSIN,55,TREMPEALEAU,121,FARM OPERATIONS - NUMBER OF OPERATIONS,"1,744"
36348,CENSUS,2002,WISCONSIN,55,TREMPEALEAU,121,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","4,328,000"


In [15]:
#making a FIPS column 
coa['FIPS'] = coa['State ANSI'] + coa['County ANSI']

#changing year to a string for the conversion
coa['Year'] = coa['Year'].astype(str)

#and a FIPSYear column
coa['FIPSYear'] = coa['FIPS'] + coa['Year']

#and turning year back into an integer
coa['Year'] = coa['Year'].astype(int)
coa

,Program,Year,State,State ANSI,County,County ANSI,Data Item,Value,FIPS,FIPSYear
0,CENSUS,2017,ARKANSAS,05,CONWAY,029,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $","1,395,000",05029,050292017
1,CENSUS,2017,ARKANSAS,05,CONWAY,029,CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,375,05029,050292017
2,CENSUS,2017,ARKANSAS,05,CONWAY,029,FARM OPERATIONS - ACRES OPERATED,"171,893",05029,050292017
3,CENSUS,2017,ARKANSAS,05,CONWAY,029,FARM OPERATIONS - NUMBER OF OPERATIONS,768,05029,050292017
4,CENSUS,2017,ARKANSAS,05,CONWAY,029,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","2,350,000",05029,050292017
...,...,...,...,...,...,...,...,...,...,...
36345,CENSUS,2002,WISCONSIN,55,TREMPEALEAU,121,CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,671,55121,551212002
36346,CENSUS,2002,WISCONSIN,55,TREMPEALEAU,121,FARM OPERATIONS - ACRES OPERATED,"367,830",55121,551212002
36347,CENSUS,2002,WISCONSIN,55,TREMPEALEAU,121,FARM OPERATIONS - NUMBER OF OPERATIONS,"1,744",55121,551212002
36348,CENSUS,2002,WISCONSIN,55,TREMPEALEAU,121,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","4,328,000",55121,551212002


Now we can spread the **Data Item** and **Value** columns into separate columns, using the FIPSYEar column we just created as an index to merge on. We can check for NaN values in the current dataframe and if we have any we can note them and check on them after our big conversion.

In [16]:
#checking for nulls
coa.isnull().sum()

Program        0
Year           0
State          0
State ANSI     0
County         0
County ANSI    0
Data Item      0
Value          0
FIPS           0
FIPSYear       0
dtype: int64

In [17]:
#spreading our dataframe, we should get about 6,000 columns
coa_spread = spread(df=coa,
       index='FIPSYear',
       columns='Data Item',
       values='Value')
coa_spread

,FIPSYear,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $",CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,FARM OPERATIONS - ACRES OPERATED,FARM OPERATIONS - NUMBER OF OPERATIONS,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - OPERATIONS WITH EXPENSE"
0,050012002,"9,640,000",304,"389,894",502,"13,387,000",409
1,050012007,"16,180,000",276,"405,103",539,"27,586,000",349
2,050012012,"26,086,000",292,"402,159",492,"46,333,000",279
3,050012017,"29,196,000",291,"414,354",488,"32,662,000",292
4,050032002,"9,987,000",122,"166,907",336,"3,792,000",180
...,...,...,...,...,...,...,...
6059,551392017,"4,455,000",486,"162,052",957,"8,229,000",523
6060,551412002,"1,877,000",543,"228,050","1,108","3,203,000",588
6061,551412007,"2,794,000",575,"221,962","1,114","6,510,000",708
6062,551412012,"4,209,000",665,"222,730","1,067","12,185,000",694


In [18]:
coa = coa.merge(coa_spread, on='FIPSYear')
coa

,Program,Year,State,State ANSI,County,County ANSI,Data Item,Value,FIPS,FIPSYear,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $",CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,FARM OPERATIONS - ACRES OPERATED,FARM OPERATIONS - NUMBER OF OPERATIONS,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - OPERATIONS WITH EXPENSE"
0,CENSUS,2017,ARKANSAS,05,CONWAY,029,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $","1,395,000",05029,050292017,"1,395,000",375,"171,893",768,"2,350,000",405
1,CENSUS,2017,ARKANSAS,05,CONWAY,029,CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,375,05029,050292017,"1,395,000",375,"171,893",768,"2,350,000",405
2,CENSUS,2017,ARKANSAS,05,CONWAY,029,FARM OPERATIONS - ACRES OPERATED,"171,893",05029,050292017,"1,395,000",375,"171,893",768,"2,350,000",405
3,CENSUS,2017,ARKANSAS,05,CONWAY,029,FARM OPERATIONS - NUMBER OF OPERATIONS,768,05029,050292017,"1,395,000",375,"171,893",768,"2,350,000",405
4,CENSUS,2017,ARKANSAS,05,CONWAY,029,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","2,350,000",05029,050292017,"1,395,000",375,"171,893",768,"2,350,000",405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36345,CENSUS,2002,WISCONSIN,55,TREMPEALEAU,121,CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,671,55121,551212002,"2,978,000",671,"367,830","1,744","4,328,000",789
36346,CENSUS,2002,WISCONSIN,55,TREMPEALEAU,121,FARM OPERATIONS - ACRES OPERATED,"367,830",55121,551212002,"2,978,000",671,"367,830","1,744","4,328,000",789
36347,CENSUS,2002,WISCONSIN,55,TREMPEALEAU,121,FARM OPERATIONS - NUMBER OF OPERATIONS,"1,744",55121,551212002,"2,978,000",671,"367,830","1,744","4,328,000",789
36348,CENSUS,2002,WISCONSIN,55,TREMPEALEAU,121,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","4,328,000",55121,551212002,"2,978,000",671,"367,830","1,744","4,328,000",789


In [19]:
#we had zero nulls, so in theory we should still have zero
#if not, we can check the rows out after dropping duplicates
coa.isnull().sum()

Program                                                                        0
Year                                                                           0
State                                                                          0
State ANSI                                                                     0
County                                                                         0
County ANSI                                                                    0
Data Item                                                                      0
Value                                                                          0
FIPS                                                                           0
FIPSYear                                                                       0
CHEMICAL TOTALS - EXPENSE, MEASURED IN $                                      26
CHEMICAL TOTALS - OPERATIONS WITH EXPENSE                                     26
FARM OPERATIONS - ACRES OPER

In [20]:
#dropping the State ANSI, County ANSI, Data Item and Value columns
coa = coa.drop(columns=['State ANSI', 'County ANSI', 'Data Item', 'Value'])
coa

,Program,Year,State,County,FIPS,FIPSYear,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $",CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,FARM OPERATIONS - ACRES OPERATED,FARM OPERATIONS - NUMBER OF OPERATIONS,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - OPERATIONS WITH EXPENSE"
0,CENSUS,2017,ARKANSAS,CONWAY,05029,050292017,"1,395,000",375,"171,893",768,"2,350,000",405
1,CENSUS,2017,ARKANSAS,CONWAY,05029,050292017,"1,395,000",375,"171,893",768,"2,350,000",405
2,CENSUS,2017,ARKANSAS,CONWAY,05029,050292017,"1,395,000",375,"171,893",768,"2,350,000",405
3,CENSUS,2017,ARKANSAS,CONWAY,05029,050292017,"1,395,000",375,"171,893",768,"2,350,000",405
4,CENSUS,2017,ARKANSAS,CONWAY,05029,050292017,"1,395,000",375,"171,893",768,"2,350,000",405
...,...,...,...,...,...,...,...,...,...,...,...,...
36345,CENSUS,2002,WISCONSIN,TREMPEALEAU,55121,551212002,"2,978,000",671,"367,830","1,744","4,328,000",789
36346,CENSUS,2002,WISCONSIN,TREMPEALEAU,55121,551212002,"2,978,000",671,"367,830","1,744","4,328,000",789
36347,CENSUS,2002,WISCONSIN,TREMPEALEAU,55121,551212002,"2,978,000",671,"367,830","1,744","4,328,000",789
36348,CENSUS,2002,WISCONSIN,TREMPEALEAU,55121,551212002,"2,978,000",671,"367,830","1,744","4,328,000",789


In [21]:
#checking for duplicates prior to dropping them
#should be most of the dataframe, about 30,000
coa.duplicated().sum()

30286

In [22]:
#dropping the dupes
coa = coa.drop_duplicates(keep='first', ignore_index=True)
coa

,Program,Year,State,County,FIPS,FIPSYear,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $",CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,FARM OPERATIONS - ACRES OPERATED,FARM OPERATIONS - NUMBER OF OPERATIONS,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - OPERATIONS WITH EXPENSE"
0,CENSUS,2017,ARKANSAS,CONWAY,05029,050292017,"1,395,000",375,"171,893",768,"2,350,000",405
1,CENSUS,2017,ARKANSAS,FAULKNER,05045,050452017,"921,000",388,"200,539","1,191","1,559,000",419
2,CENSUS,2017,ARKANSAS,GARLAND,05051,050512017,"86,000",104,"33,950",357,"214,000",98
3,CENSUS,2017,ARKANSAS,GRANT,05053,050532017,"91,000",102,"49,218",281,"191,000",118
4,CENSUS,2017,ARKANSAS,HOT SPRING,05059,050592017,"167,000",186,"77,331",563,"243,000",181
...,...,...,...,...,...,...,...,...,...,...,...,...
6059,CENSUS,2002,WISCONSIN,MONROE,55081,550812002,"2,396,000",817,"351,775","1,938","3,979,000","1,037"
6060,CENSUS,2002,WISCONSIN,PEPIN,55091,550912002,"1,188,000",298,"111,313",501,"1,478,000",310
6061,CENSUS,2002,WISCONSIN,PIERCE,55093,550932002,"2,368,000",659,"267,311","1,510","4,014,000",691
6062,CENSUS,2002,WISCONSIN,ST CROIX,55109,551092002,"2,931,000",701,"310,178","1,864","4,506,000",773


In [23]:
#and renaming the variable columns to something more reasonable
coa = coa.rename(columns={'CHEMICAL TOTALS - EXPENSE, MEASURED IN $':'Chem_expense_dollars',
                   'CHEMICAL TOTALS - OPERATIONS WITH EXPENSE':'Chem_operations',
                   'FARM OPERATIONS - ACRES OPERATED':'Acres_operated',
                   'FARM OPERATIONS - NUMBER OF OPERATIONS':'Operations',
                   'FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $':'Fert_expense_dollars',
                   'FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - OPERATIONS WITH EXPENSE':'Fert_operations'})
coa

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
0,CENSUS,2017,ARKANSAS,CONWAY,05029,050292017,"1,395,000",375,"171,893",768,"2,350,000",405
1,CENSUS,2017,ARKANSAS,FAULKNER,05045,050452017,"921,000",388,"200,539","1,191","1,559,000",419
2,CENSUS,2017,ARKANSAS,GARLAND,05051,050512017,"86,000",104,"33,950",357,"214,000",98
3,CENSUS,2017,ARKANSAS,GRANT,05053,050532017,"91,000",102,"49,218",281,"191,000",118
4,CENSUS,2017,ARKANSAS,HOT SPRING,05059,050592017,"167,000",186,"77,331",563,"243,000",181
...,...,...,...,...,...,...,...,...,...,...,...,...
6059,CENSUS,2002,WISCONSIN,MONROE,55081,550812002,"2,396,000",817,"351,775","1,938","3,979,000","1,037"
6060,CENSUS,2002,WISCONSIN,PEPIN,55091,550912002,"1,188,000",298,"111,313",501,"1,478,000",310
6061,CENSUS,2002,WISCONSIN,PIERCE,55093,550932002,"2,368,000",659,"267,311","1,510","4,014,000",691
6062,CENSUS,2002,WISCONSIN,ST CROIX,55109,551092002,"2,931,000",701,"310,178","1,864","4,506,000",773


In [24]:
coa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6064 entries, 0 to 6063
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Program               6064 non-null   object
 1   Year                  6064 non-null   int32 
 2   State                 6064 non-null   object
 3   County                6064 non-null   object
 4   FIPS                  6064 non-null   object
 5   FIPSYear              6064 non-null   object
 6   Chem_expense_dollars  6055 non-null   object
 7   Chem_operations       6055 non-null   object
 8   Acres_operated        6064 non-null   object
 9   Operations            6064 non-null   object
 10  Fert_expense_dollars  6056 non-null   object
 11  Fert_operations       6056 non-null   object
dtypes: int32(1), object(11)
memory usage: 544.9+ KB


We are back down to 6,064 rows and our variables are spread out across 6 columns with short (but still descriptive) column names! Before we call it a day and move on to reading in our State-level data for the other 16 year, lets check the null values we saw come up a moment ago.

In [25]:
coa.isnull().sum()

Program                 0
Year                    0
State                   0
County                  0
FIPS                    0
FIPSYear                0
Chem_expense_dollars    9
Chem_operations         9
Acres_operated          0
Operations              0
Fert_expense_dollars    8
Fert_operations         8
dtype: int64

In [26]:
#checking the nans for Chem_expense_dollars
coa[coa['Chem_expense_dollars'].isnull()==True]

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
1385,CENSUS,2017,TEXAS,LOVING,48301,483012017,NaN,NaN,"468,140",8,NaN,NaN
1598,CENSUS,2012,CALIFORNIA,SAN FRANCISCO,06075,060752012,NaN,NaN,12,6,NaN,NaN
1627,CENSUS,2012,CALIFORNIA,ALPINE,06003,060032012,NaN,NaN,(D),3,(D),1
1854,CENSUS,2012,GEORGIA,CHATTAHOOCHEE,13053,130532012,NaN,NaN,"4,081",13,(D),2
2990,CENSUS,2012,WISCONSIN,MENOMINEE,55078,550782012,NaN,NaN,561,5,NaN,NaN
3821,CENSUS,2007,MINNESOTA,COOK,27031,270312007,NaN,NaN,"2,402",13,(D),1
4417,CENSUS,2007,TEXAS,LOVING,48301,483012007,NaN,NaN,"426,792",9,NaN,NaN
4506,CENSUS,2007,WISCONSIN,MENOMINEE,55078,550782007,NaN,NaN,318,4,NaN,NaN
6022,CENSUS,2002,WISCONSIN,MENOMINEE,55078,550782002,NaN,NaN,354,4,(D),2


In [27]:
#checking the nans for Fert_expense_dollars
coa[coa['Fert_expense_dollars'].isnull()==True]

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
1385,CENSUS,2017,TEXAS,LOVING,48301,483012017,NaN,NaN,"468,140",8,NaN,NaN
1598,CENSUS,2012,CALIFORNIA,SAN FRANCISCO,06075,060752012,NaN,NaN,12,6,NaN,NaN
2901,CENSUS,2012,TEXAS,LOVING,48301,483012012,(D),1,"379,524",10,NaN,NaN
2990,CENSUS,2012,WISCONSIN,MENOMINEE,55078,550782012,NaN,NaN,561,5,NaN,NaN
3143,CENSUS,2007,CALIFORNIA,ALPINE,06003,060032007,(D),1,"1,810",7,NaN,NaN
4417,CENSUS,2007,TEXAS,LOVING,48301,483012007,NaN,NaN,"426,792",9,NaN,NaN
4506,CENSUS,2007,WISCONSIN,MENOMINEE,55078,550782007,NaN,NaN,318,4,NaN,NaN
5933,CENSUS,2002,TEXAS,LOVING,48301,483012002,(D),1,"515,192",14,NaN,NaN


There are so few NaN values in our data that they should not be a burden to impute, especially knowing that we can check the USDA data for these specific counties. We have some missing data with '(D)' as the value. Per the USDA's glossary, this indicates: Withheld to avoid disclosing data for individual operations. When we bring in our state-wide data we can attempt to backfill these '(D)' values.

We also could have some values with '(Z)' which from the glossary means: Less than half the rounding unit.
The number of operators and the acres operated all are down to the single digit, but the expense columns are rounded to the nearest 1,000 value. In other words, a (Z) value would indicate that the county have less than $500 in spending on that expense. We can fill these values with 0.

These values wont come up with normal row slicing methods (the .loc function and slicing the dataframe where **Chem_expense_dollars** == to '(D)' did not work). We'll need to work on these string values a bit before we can actually find all of them in the dataframe, it seems...

In [28]:
#removing the parenthesis from the D and Z values
coa = coa.replace(to_replace='\(', value='', regex=True)
coa = coa.replace(to_replace='\)', value='', regex=True)
coa

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
0,CENSUS,2017,ARKANSAS,CONWAY,05029,050292017,"1,395,000",375,"171,893",768,"2,350,000",405
1,CENSUS,2017,ARKANSAS,FAULKNER,05045,050452017,"921,000",388,"200,539","1,191","1,559,000",419
2,CENSUS,2017,ARKANSAS,GARLAND,05051,050512017,"86,000",104,"33,950",357,"214,000",98
3,CENSUS,2017,ARKANSAS,GRANT,05053,050532017,"91,000",102,"49,218",281,"191,000",118
4,CENSUS,2017,ARKANSAS,HOT SPRING,05059,050592017,"167,000",186,"77,331",563,"243,000",181
...,...,...,...,...,...,...,...,...,...,...,...,...
6059,CENSUS,2002,WISCONSIN,MONROE,55081,550812002,"2,396,000",817,"351,775","1,938","3,979,000","1,037"
6060,CENSUS,2002,WISCONSIN,PEPIN,55091,550912002,"1,188,000",298,"111,313",501,"1,478,000",310
6061,CENSUS,2002,WISCONSIN,PIERCE,55093,550932002,"2,368,000",659,"267,311","1,510","4,014,000",691
6062,CENSUS,2002,WISCONSIN,ST CROIX,55109,551092002,"2,931,000",701,"310,178","1,864","4,506,000",773


In [29]:
#checking how the 'd' values are displayed... there may be a leading space?
coa['Fert_expense_dollars'].value_counts()

 D            54
199,000        7
509,000        7
143,000        5
777,000        5
              ..
1,891,000      1
20,494,000     1
12,044,000     1
43,521,000     1
4,506,000      1
Name: Fert_expense_dollars, Length: 4900, dtype: int64

In [30]:
#trying to find them with a leading space, ' D'
coa[coa['Chem_expense_dollars'] == ' D']

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
91,CENSUS,2017,CALIFORNIA,DEL NORTE,06015,060152017,D,23,"19,770",90,"548,000",23
111,CENSUS,2017,CALIFORNIA,ALPINE,06003,060032017,D,2,"3,173",6,D,1
121,CENSUS,2017,CALIFORNIA,TUOLUMNE,06109,061092017,D,103,"122,539",417,"739,000",77
243,CENSUS,2017,GEORGIA,COBB,13067,130672017,D,40,"2,524",116,"80,000",49
245,CENSUS,2017,GEORGIA,DE KALB,13089,130892017,D,5,468,34,"44,000",16
...,...,...,...,...,...,...,...,...,...,...,...,...
5928,CENSUS,2002,TEXAS,CULBERSON,48109,481092002,D,12,"1,694,512",74,"249,000",10
5933,CENSUS,2002,TEXAS,LOVING,48301,483012002,D,1,"515,192",14,NaN,NaN
5935,CENSUS,2002,TEXAS,PRESIDIO,48377,483772002,D,18,"1,503,639",123,D,29
5937,CENSUS,2002,TEXAS,TERRELL,48443,484432002,D,8,"1,413,092",76,D,3


In [31]:
# checking for withheld 'D' data in all 6 variables
coa_D_rows = coa[(coa['Chem_expense_dollars'] == ' D') |
    (coa['Chem_operations'] == ' D') |
    (coa['Acres_operated'] == ' D') |
    (coa['Operations'] == ' D') |
    (coa['Fert_expense_dollars'] == ' D') | 
    (coa['Fert_operations'] == ' D')]
coa_D_rows

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
91,CENSUS,2017,CALIFORNIA,DEL NORTE,06015,060152017,D,23,"19,770",90,"548,000",23
92,CENSUS,2017,CALIFORNIA,HUMBOLDT,06023,060232017,"674,000",123,"621,043",849,D,253
111,CENSUS,2017,CALIFORNIA,ALPINE,06003,060032017,D,2,"3,173",6,D,1
112,CENSUS,2017,CALIFORNIA,AMADOR,06005,060052017,"1,322,000",227,"181,492",482,D,182
121,CENSUS,2017,CALIFORNIA,TUOLUMNE,06109,061092017,D,103,"122,539",417,"739,000",77
...,...,...,...,...,...,...,...,...,...,...,...,...
5937,CENSUS,2002,TEXAS,TERRELL,48443,484432002,D,8,"1,413,092",76,D,3
5938,CENSUS,2002,TEXAS,WARD,48475,484752002,D,7,"465,639",86,"17,000",14
5939,CENSUS,2002,TEXAS,WINKLER,48495,484952002,"1,000",7,"491,718",44,D,10
6011,CENSUS,2002,WISCONSIN,IRON,55051,550512002,"25,000",13,"12,741",62,D,28


In [32]:
# checking for the ' Z' values
coa[(coa['Chem_expense_dollars'] == ' Z') |
    (coa['Chem_operations'] == ' Z') |
    (coa['Acres_operated'] == ' Z') |
    (coa['Operations'] == ' Z') |
    (coa['Fert_expense_dollars'] == ' Z') | 
    (coa['Fert_operations'] == ' Z')]

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
2305,CENSUS,2012,MINNESOTA,COOK,27031,270312012,Z,3,"2,272",18,"1,000",6
5337,CENSUS,2002,MINNESOTA,COOK,27031,270312002,Z,3,"3,269",20,D,3


In [33]:
#adding 2 to the shape of for the 
print(f'The intentionally withheld or rounded down values make up {round((coa_D_rows.shape[0]+2)/coa.shape[0]*100 ,2)}% of our data.')

The intentionally withheld or rounded down values make up 2.26% of our data.


In [34]:
#replacing the two 'Z' values we found with zeroes
coa = coa.replace(to_replace=' Z', value='0', regex=True)
coa[coa['Chem_expense_dollars'] == '0']

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
2305,CENSUS,2012,MINNESOTA,COOK,27031,270312012,0,3,"2,272",18,"1,000",6
5337,CENSUS,2002,MINNESOTA,COOK,27031,270312002,0,3,"3,269",20,D,3


We are able to track down both the rows with *intentionally* missing data (' D'), the two rows where the value is effectively 0, and the plain-old missing data ('NaN'). We imputed the 'Z' values with a 0 - only two rows were affected. 

We could impute the remaining data in a couple ways. We should be able to get the total dollars, acres, and operations at the state level for each of these years. With that data, we can extrapolate a reasonable estimate for either the NaN or ' D' values and impute it.

We know from checking the null values that they are limited to Texas, California, Georgia, Wisconsin, and Minnesota.

Let's see which states come up in the dataframe holding all of the 'D rows' and if the issue is spread across all four years or not.

In [35]:
coa_D_rows['State'].value_counts()

TEXAS             46
GEORGIA           22
FLORIDA           17
CALIFORNIA        16
NORTH CAROLINA    13
MISSOURI           7
MINNESOTA          6
WISCONSIN          6
NEBRASKA           2
Name: State, dtype: int64

In [36]:
coa_D_rows['Year'].value_counts()

2002    41
2012    38
2017    33
2007    23
Name: Year, dtype: int64

We are missing data from 9 state across all 4 years of our Census data. Because we will be replacing the NaN and withheld values in the same way, we can convert our ' D' values to Nans.

In [37]:
#replacing the D values with nulls
coa[['Chem_expense_dollars',
     'Chem_operations',
     'Acres_operated',
     'Operations',
     'Fert_expense_dollars',
     'Fert_operations']] = coa[['Chem_expense_dollars',
                                'Chem_operations',
                                'Acres_operated',
                                'Operations',
                                'Fert_expense_dollars',
                                'Fert_operations']].replace(' D', np.NaN)

In [38]:
#checking that the replacement worked
coa[coa['Chem_expense_dollars']== ' D']

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations


We will circle back to the USDA database and pull state level data for these 6 variables and use that to help us impute data.

In [39]:
#pulling in the state level data for the 9 states in the census with missing data
coa_state = pd.read_csv('data/coa_state_level.csv', 
                        thousands=',',
                        header=0,
                        names=['Program', 'Year', 'Period','Week Ending', 'Geo Level', 'State', 'State ANSI', 'Ag District', 'Ag District Code', 'County','County ANSI', 'Zip Code', 'Region','watershed_code', 'Watershed', 'Commodity', 'Data Item', 'Domain', 'Domain Category', 'Value', 'CV (%)'],
                        usecols=['Program', 'Year', 'State', 'State ANSI', 'County','County ANSI', 'Data Item', 'Value'])
coa_state

,Program,Year,State,State ANSI,County,County ANSI,Data Item,Value
0,CENSUS,2017,CALIFORNIA,6,NaN,NaN,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $",2409928000
1,CENSUS,2017,CALIFORNIA,6,NaN,NaN,CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,37794
2,CENSUS,2017,CALIFORNIA,6,NaN,NaN,FARM OPERATIONS - ACRES OPERATED,24522801
3,CENSUS,2017,CALIFORNIA,6,NaN,NaN,FARM OPERATIONS - NUMBER OF OPERATIONS,70521
4,CENSUS,2017,CALIFORNIA,6,NaN,NaN,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $",2082908000
...,...,...,...,...,...,...,...,...
211,CENSUS,2002,WISCONSIN,55,NaN,NaN,CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,37052
212,CENSUS,2002,WISCONSIN,55,NaN,NaN,FARM OPERATIONS - ACRES OPERATED,15741552
213,CENSUS,2002,WISCONSIN,55,NaN,NaN,FARM OPERATIONS - NUMBER OF OPERATIONS,77131
214,CENSUS,2002,WISCONSIN,55,NaN,NaN,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $",260006000


We can transform this small dataframe in the same way we tidied the census data, pivoting the variables into discrete columns to make aggregating simpler.

In [40]:
#checking for duplicate ansi numbers
coa_state['State ANSI'].value_counts()

6     24
12    24
13    24
27    24
29    24
31    24
37    24
48    24
55    24
Name: State ANSI, dtype: int64

In [41]:
#converting state ANSI to string
coa_state['State ANSI'] = coa_state['State ANSI'].astype(str)
#changing year to a string for the conversion
coa_state['Year'] = coa_state['Year'].astype(str)

#making a YearANSI column 
coa_state['YearANSI'] = coa_state['Year'] + coa_state['State ANSI']

#turning year back into an integer
coa_state['Year'] = coa_state['Year'].astype(int)

#spreading our short dataframe
coa_state_spread = spread(df=coa_state,
       index='YearANSI',
       columns='Data Item',
       values='Value')
#and seeing what we get
coa_state_spread

,YearANSI,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $",CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,FARM OPERATIONS - ACRES OPERATED,FARM OPERATIONS - NUMBER OF OPERATIONS,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - OPERATIONS WITH EXPENSE"
0,200212,321442000,21458,10414877,44081,289115000,26563
1,200213,197468000,20872,10744239,49311,190338000,27037
2,200227,350354000,38940,27512270,80839,493994000,44286
3,200229,190396000,35192,29946035,106797,355973000,63363
4,200231,281683000,29632,45903116,49355,465636000,32982
5,200237,220109000,26680,9079001,53930,238389000,36121
6,200248,345210000,85665,129877666,228926,555239000,108277
7,200255,184368000,37052,15741552,77131,260006000,42048
8,20026,1102974000,46868,27589027,79631,834536000,50120
9,200712,376699000,20462,9231570,47463,439145000,27041


We get 36 rows once we translate the variables into discrete columns. Next, to combine this with the coa_state dataframe so we can sort by state more easily.

In [42]:
#merging the state dataframe with the spread state dataframe
coa_state = coa_state.merge(coa_state_spread, on='YearANSI')

#dropping the Data item and values columns so we can get rid of the duplicate values
coa_state = coa_state.drop(columns=['Data Item', 'Value'])

#dropping the dupes from the spread-state dataframe
coa_state = coa_state.drop_duplicates(keep='first', ignore_index=True)
coa_state

,Program,Year,State,State ANSI,County,County ANSI,YearANSI,"CHEMICAL TOTALS - EXPENSE, MEASURED IN $",CHEMICAL TOTALS - OPERATIONS WITH EXPENSE,FARM OPERATIONS - ACRES OPERATED,FARM OPERATIONS - NUMBER OF OPERATIONS,"FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $","FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - OPERATIONS WITH EXPENSE"
0,CENSUS,2017,CALIFORNIA,6,NaN,NaN,20176,2409928000,37794,24522801,70521,2082908000,39225
1,CENSUS,2017,FLORIDA,12,NaN,NaN,201712,544350000,17806,9731731,47590,583131000,22004
2,CENSUS,2017,GEORGIA,13,NaN,NaN,201713,410369000,20244,9953730,42439,452329000,24684
3,CENSUS,2017,MINNESOTA,27,NaN,NaN,201727,797679000,38474,25516982,68822,1374497000,40476
4,CENSUS,2017,MISSOURI,29,NaN,NaN,201729,601022000,37082,27781883,95320,854116000,50995
5,CENSUS,2017,NEBRASKA,31,NaN,NaN,201731,861588000,29262,44986821,46332,1246147000,29078
6,CENSUS,2017,NORTH CAROLINA,37,NaN,NaN,201737,378720000,21869,8430522,46418,478251000,28664
7,CENSUS,2017,TEXAS,48,NaN,NaN,201748,750063000,90626,127036184,248416,1050689000,91760
8,CENSUS,2017,WISCONSIN,55,NaN,NaN,201755,355396000,32226,14318630,64793,719654000,37160
9,CENSUS,2012,CALIFORNIA,6,NaN,NaN,20126,2190674000,44536,25569001,77857,1806062000,38958


We can start by checking our the original dataframe to see where we have the most and least null values.

In [43]:
coa.isnull().sum()

Program                   0
Year                      0
State                     0
County                    0
FIPS                      0
FIPSYear                  0
Chem_expense_dollars    109
Chem_operations           9
Acres_operated           18
Operations                0
Fert_expense_dollars     62
Fert_operations           8
dtype: int64

We do not have any missing values for the **Operations** variable, so we can use this as our key. We will need to convert the variables to numeric, and then fill the values in by state. Because there were commas in the number data for all of the variable columns, now might be a good time to reimport the dataframe with the 'thousands' parameter in the read_csv function...

In [44]:
#sending it to a CSV file 
#coa.to_csv('coaRough.csv', index=False)

In [45]:
#letting thousands do our cleanup for us
#...trading 'pythonic' code for expedient code
coa = pd.read_csv('coaRough.csv', thousands=',')
coa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6064 entries, 0 to 6063
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Program               6064 non-null   object 
 1   Year                  6064 non-null   int64  
 2   State                 6064 non-null   object 
 3   County                6064 non-null   object 
 4   FIPS                  6064 non-null   int64  
 5   FIPSYear              6064 non-null   int64  
 6   Chem_expense_dollars  5955 non-null   float64
 7   Chem_operations       6055 non-null   float64
 8   Acres_operated        6046 non-null   float64
 9   Operations            6064 non-null   int64  
 10  Fert_expense_dollars  6002 non-null   float64
 11  Fert_operations       6056 non-null   float64
dtypes: float64(5), int64(4), object(3)
memory usage: 568.6+ KB


In [46]:
coa.isnull().sum()

Program                   0
Year                      0
State                     0
County                    0
FIPS                      0
FIPSYear                  0
Chem_expense_dollars    109
Chem_operations           9
Acres_operated           18
Operations                0
Fert_expense_dollars     62
Fert_operations           8
dtype: int64

We didn't gain or lose any NaN's with the reimport. Imputing these missing values may become tedious, but all of the values we can impute *now* with values we are confident in will translate to much cleaner data when we combine our datasets. Each county will have 52 rows of dat per year when we combine dataframes.

Here's our gameplan for filling in these missing values:

* Fill the Fert and Chem operations columns with the average for that county across all years
    If no value is listed for that county for any other year, impute 1

* Fill the Acres_Operated by multiplying the number of Operators by the average number of acres per operator for that state and year

* Fill the Fert and Chem Expenses by multiplying the number of Operators by the average expense 

First will need to find the counties with missing values in Chem_operations and Fert_operations. For each county that we find with null values we can 

In [47]:
coa[coa['Chem_operations'].isnull()]

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
1385,CENSUS,2017,TEXAS,LOVING,48301,483012017,NaN,NaN,468140.0,8,NaN,NaN
1598,CENSUS,2012,CALIFORNIA,SAN FRANCISCO,6075,60752012,NaN,NaN,12.0,6,NaN,NaN
1627,CENSUS,2012,CALIFORNIA,ALPINE,6003,60032012,NaN,NaN,NaN,3,NaN,1.0
1854,CENSUS,2012,GEORGIA,CHATTAHOOCHEE,13053,130532012,NaN,NaN,4081.0,13,NaN,2.0
2990,CENSUS,2012,WISCONSIN,MENOMINEE,55078,550782012,NaN,NaN,561.0,5,NaN,NaN
3821,CENSUS,2007,MINNESOTA,COOK,27031,270312007,NaN,NaN,2402.0,13,NaN,1.0
4417,CENSUS,2007,TEXAS,LOVING,48301,483012007,NaN,NaN,426792.0,9,NaN,NaN
4506,CENSUS,2007,WISCONSIN,MENOMINEE,55078,550782007,NaN,NaN,318.0,4,NaN,NaN
6022,CENSUS,2002,WISCONSIN,MENOMINEE,55078,550782002,NaN,NaN,354.0,4,NaN,2.0


We are missing values in Loving, Texas for 2007 and 2017, let's see what we have for that county in other years.

In [48]:
#we'll be repeating this quite a bit, lets make sure the code works and then make some functions
#inspiration for these null fixes came from: 
# https://towardsdatascience.com/pandas-tricks-for-imputing-missing-data-63da3d14c0d6

coa.loc[(coa['State']=='TEXAS') & (coa['County']=='LOVING')].Chem_operations.mean()

1.0

In [49]:
def st_cty_filter(state, county):
    #filtering by state and county with .loc
    return coa.loc[(coa['State']==state) & (coa['County']==county)]

In [50]:
#test drive #1
st_cty_filter('TEXAS', 'LOVING') 

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
1385,CENSUS,2017,TEXAS,LOVING,48301,483012017,NaN,NaN,468140.0,8,NaN,NaN
2901,CENSUS,2012,TEXAS,LOVING,48301,483012012,NaN,1.0,379524.0,10,NaN,NaN
4417,CENSUS,2007,TEXAS,LOVING,48301,483012007,NaN,NaN,426792.0,9,NaN,NaN
5933,CENSUS,2002,TEXAS,LOVING,48301,483012002,NaN,1.0,515192.0,14,NaN,NaN


In [51]:
def mean_ops(state, county, operation_type):
    #save some copy/pasting time and effort when finding the mean number of Chem or Fert operations
    thingy = coa.loc[(coa['State']==state) & (coa['County']==county)]
    return round(thingy[operation_type].mean())

In [52]:
#test drive #2
mean_ops('TEXAS', 'LOVING', 'Chem_operations')

1

Let's make sure the replacement works

In [53]:
coa.loc[(coa['State']=='TEXAS') &
        (coa['County']=='LOVING') &
        (coa['Chem_operations'].isnull()), 'Chem_operations'] = mean_ops('TEXAS', 'LOVING', 'Chem_operations')

In [54]:
coa[coa['Chem_operations'].isnull()]

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
1598,CENSUS,2012,CALIFORNIA,SAN FRANCISCO,6075,60752012,NaN,NaN,12.0,6,NaN,NaN
1627,CENSUS,2012,CALIFORNIA,ALPINE,6003,60032012,NaN,NaN,NaN,3,NaN,1.0
1854,CENSUS,2012,GEORGIA,CHATTAHOOCHEE,13053,130532012,NaN,NaN,4081.0,13,NaN,2.0
2990,CENSUS,2012,WISCONSIN,MENOMINEE,55078,550782012,NaN,NaN,561.0,5,NaN,NaN
3821,CENSUS,2007,MINNESOTA,COOK,27031,270312007,NaN,NaN,2402.0,13,NaN,1.0
4506,CENSUS,2007,WISCONSIN,MENOMINEE,55078,550782007,NaN,NaN,318.0,4,NaN,NaN
6022,CENSUS,2002,WISCONSIN,MENOMINEE,55078,550782002,NaN,NaN,354.0,4,NaN,2.0


In [55]:
coa.loc[(coa['State']=='CALIFORNIA') &
        (coa['County']=='ALPINE') &
        (coa['Chem_operations'].isnull()), 'Chem_operations'] = mean_ops('CALIFORNIA', 'ALPINE', 'Chem_operations')

In [56]:
coa[coa['Chem_operations'].isnull()]

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
1598,CENSUS,2012,CALIFORNIA,SAN FRANCISCO,6075,60752012,NaN,NaN,12.0,6,NaN,NaN
1854,CENSUS,2012,GEORGIA,CHATTAHOOCHEE,13053,130532012,NaN,NaN,4081.0,13,NaN,2.0
2990,CENSUS,2012,WISCONSIN,MENOMINEE,55078,550782012,NaN,NaN,561.0,5,NaN,NaN
3821,CENSUS,2007,MINNESOTA,COOK,27031,270312007,NaN,NaN,2402.0,13,NaN,1.0
4506,CENSUS,2007,WISCONSIN,MENOMINEE,55078,550782007,NaN,NaN,318.0,4,NaN,NaN
6022,CENSUS,2002,WISCONSIN,MENOMINEE,55078,550782002,NaN,NaN,354.0,4,NaN,2.0


In [57]:
coa.loc[(coa['State']=='CALIFORNIA') &
        (coa['County']=='SAN FRANCISCO') &
        (coa['Chem_operations'].isnull()), 'Chem_operations'] = mean_ops('CALIFORNIA', 'SAN FRANCISCO', 'Chem_operations')

In [58]:
coa[coa['Chem_operations'].isnull()]

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
1854,CENSUS,2012,GEORGIA,CHATTAHOOCHEE,13053,130532012,NaN,NaN,4081.0,13,NaN,2.0
2990,CENSUS,2012,WISCONSIN,MENOMINEE,55078,550782012,NaN,NaN,561.0,5,NaN,NaN
3821,CENSUS,2007,MINNESOTA,COOK,27031,270312007,NaN,NaN,2402.0,13,NaN,1.0
4506,CENSUS,2007,WISCONSIN,MENOMINEE,55078,550782007,NaN,NaN,318.0,4,NaN,NaN
6022,CENSUS,2002,WISCONSIN,MENOMINEE,55078,550782002,NaN,NaN,354.0,4,NaN,2.0


In [59]:
coa.loc[(coa['State']=='GEORGIA') &
        (coa['County']=='CHATTAHOOCHEE') &
        (coa['Chem_operations'].isnull()), 'Chem_operations'] = mean_ops('GEORGIA', 'CHATTAHOOCHEE', 'Chem_operations')

In [60]:
coa[coa['Chem_operations'].isnull()]

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
2990,CENSUS,2012,WISCONSIN,MENOMINEE,55078,550782012,NaN,NaN,561.0,5,NaN,NaN
3821,CENSUS,2007,MINNESOTA,COOK,27031,270312007,NaN,NaN,2402.0,13,NaN,1.0
4506,CENSUS,2007,WISCONSIN,MENOMINEE,55078,550782007,NaN,NaN,318.0,4,NaN,NaN
6022,CENSUS,2002,WISCONSIN,MENOMINEE,55078,550782002,NaN,NaN,354.0,4,NaN,2.0


In [61]:
coa.loc[(coa['State']=='WISCONSIN') &
        (coa['County']=='MENOMINEE') &
        (coa['Chem_operations'].isnull()), 'Chem_operations'] = mean_ops('WISCONSIN', 'MENOMINEE', 'Chem_operations')

In [62]:
coa[coa['Chem_operations'].isnull()]

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
3821,CENSUS,2007,MINNESOTA,COOK,27031,270312007,NaN,NaN,2402.0,13,NaN,1.0


In [63]:
coa.loc[(coa['State']=='MINNESOTA') &
        (coa['County']=='COOK') &
        (coa['Chem_operations'].isnull()), 'Chem_operations'] = mean_ops('MINNESOTA', 'COOK', 'Chem_operations')

In [64]:
coa[coa['Chem_operations'].isnull()] #last one!

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations


In [65]:
coa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6064 entries, 0 to 6063
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Program               6064 non-null   object 
 1   Year                  6064 non-null   int64  
 2   State                 6064 non-null   object 
 3   County                6064 non-null   object 
 4   FIPS                  6064 non-null   int64  
 5   FIPSYear              6064 non-null   int64  
 6   Chem_expense_dollars  5955 non-null   float64
 7   Chem_operations       6064 non-null   float64
 8   Acres_operated        6046 non-null   float64
 9   Operations            6064 non-null   int64  
 10  Fert_expense_dollars  6002 non-null   float64
 11  Fert_operations       6056 non-null   float64
dtypes: float64(5), int64(4), object(3)
memory usage: 568.6+ KB


Now we can do the same for Fert_operations. Let's see what the NaN values are there

In [66]:
coa[coa['Fert_operations'].isnull()]

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
1385,CENSUS,2017,TEXAS,LOVING,48301,483012017,NaN,1.0,468140.0,8,NaN,NaN
1598,CENSUS,2012,CALIFORNIA,SAN FRANCISCO,6075,60752012,NaN,4.0,12.0,6,NaN,NaN
2901,CENSUS,2012,TEXAS,LOVING,48301,483012012,NaN,1.0,379524.0,10,NaN,NaN
2990,CENSUS,2012,WISCONSIN,MENOMINEE,55078,550782012,NaN,1.0,561.0,5,NaN,NaN
3143,CENSUS,2007,CALIFORNIA,ALPINE,6003,60032007,NaN,1.0,1810.0,7,NaN,NaN
4417,CENSUS,2007,TEXAS,LOVING,48301,483012007,NaN,1.0,426792.0,9,NaN,NaN
4506,CENSUS,2007,WISCONSIN,MENOMINEE,55078,550782007,NaN,1.0,318.0,4,NaN,NaN
5933,CENSUS,2002,TEXAS,LOVING,48301,483012002,NaN,1.0,515192.0,14,NaN,NaN


In [67]:
#starting with Loving county, Texas--all four years are missing for the fertilizer operations
#we'll just set this one to be '0'
coa.loc[(coa['State']=='TEXAS') &
        (coa['County']=='LOVING') &
        (coa['Fert_operations'].isnull()), 'Fert_operations'] = 0

In [68]:
coa[coa['Fert_operations'].isnull()]

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
1598,CENSUS,2012,CALIFORNIA,SAN FRANCISCO,6075,60752012,NaN,4.0,12.0,6,NaN,NaN
2990,CENSUS,2012,WISCONSIN,MENOMINEE,55078,550782012,NaN,1.0,561.0,5,NaN,NaN
3143,CENSUS,2007,CALIFORNIA,ALPINE,6003,60032007,NaN,1.0,1810.0,7,NaN,NaN
4506,CENSUS,2007,WISCONSIN,MENOMINEE,55078,550782007,NaN,1.0,318.0,4,NaN,NaN


In [69]:
#Menominee WI is next
coa.loc[(coa['State']=='WISCONSIN') &
        (coa['County']=='MENOMINEE') &
        (coa['Fert_operations'].isnull()), 'Fert_operations'] = mean_ops('WISCONSIN', 'MENOMINEE', 'Fert_operations')

In [70]:
coa[coa['Fert_operations'].isnull()]

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
1598,CENSUS,2012,CALIFORNIA,SAN FRANCISCO,6075,60752012,NaN,4.0,12.0,6,NaN,NaN
3143,CENSUS,2007,CALIFORNIA,ALPINE,6003,60032007,NaN,1.0,1810.0,7,NaN,NaN


In [71]:
#San Francisco is up
coa.loc[(coa['State']=='CALIFORNIA') &
        (coa['County']=='SAN FRANCISCO') &
        (coa['Fert_operations'].isnull()), 'Fert_operations'] = mean_ops('CALIFORNIA', 'SAN FRANCISCO', 'Fert_operations')

In [72]:
coa[coa['Fert_operations'].isnull()]

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
3143,CENSUS,2007,CALIFORNIA,ALPINE,6003,60032007,NaN,1.0,1810.0,7,NaN,NaN


In [73]:
#and finally, alpine county
coa.loc[(coa['State']=='CALIFORNIA') &
        (coa['County']=='ALPINE') &
        (coa['Fert_operations'].isnull()), 'Fert_operations'] = mean_ops('CALIFORNIA', 'ALPINE', 'Fert_operations')

In [74]:
coa[coa['Fert_operations'].isnull()]

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations


Now that we have cleaned up the NaNs in the operations columns, we can perform some calculations to figure out the average number of acres per operation, the average Chemical expenses per chemical operation, and the average fertilizer expenses per fertilizer operation. 

Because we were deliberate and granular with our fixes for the three "operations" columns, we can be *much* more confident in our replacements that might be base off out imputed values.

In [75]:
#dividing the sum of the Acres_operated column by the sum of the Operations column
print(f'The average number of acres per operation is {round(coa["Acres_operated"].sum()/coa["Operations"].sum())}')

The average number of acres per operation is 395


In [76]:
#dividing the sum of the Chem_expense_dollars column by the sum of the Chem_operations column
print(f'The average chemical expenses per operation using chemicals \
is {round(coa["Chem_expense_dollars"].sum()/coa["Chem_operations"].sum())}')

The average chemical expenses per operation using chemicals is 16242


In [77]:
#dividing the sum of the Fert_expense_dollars column by the sum of the Fert_operations column
print(f'The average fertilizer expenses per operation using fertilizer \
is {round(coa["Fert_expense_dollars"].sum()/coa["Fert_operations"].sum())}')

The average fertilizer expenses per operation using fertilizer is 21400


In [78]:
#setting up a null dataframe for acres operated
acres_null = coa[coa['Acres_operated'].isnull()] 

In [79]:
#checking our work
acres_null

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
136,CENSUS,2017,FLORIDA,CLAY,12019,120192017,71000.0,96.0,NaN,361,255000.0,139.0
168,CENSUS,2017,FLORIDA,FRANKLIN,12037,120372017,2000.0,3.0,NaN,15,NaN,4.0
170,CENSUS,2017,FLORIDA,GULF,12045,120452017,8000.0,12.0,NaN,46,NaN,14.0
1474,CENSUS,2017,WISCONSIN,MENOMINEE,55078,550782017,NaN,1.0,NaN,3,NaN,1.0
1493,CENSUS,2017,WISCONSIN,MILWAUKEE,55079,550792017,128000.0,46.0,NaN,86,326000.0,41.0
1607,CENSUS,2012,CALIFORNIA,DEL NORTE,6015,60152012,NaN,22.0,NaN,121,545000.0,38.0
1627,CENSUS,2012,CALIFORNIA,ALPINE,6003,60032012,NaN,1.0,NaN,3,NaN,1.0
2593,CENSUS,2012,NORTH CAROLINA,DARE,37055,370552012,81000.0,4.0,NaN,9,215000.0,6.0
2664,CENSUS,2012,NORTH CAROLINA,SWAIN,37173,371732012,NaN,53.0,NaN,94,64000.0,54.0
4630,CENSUS,2002,CALIFORNIA,SAN FRANCISCO,6075,60752002,NaN,6.0,NaN,8,5000.0,8.0


In [80]:
#filling the nulls
acres_null['Acres_operated'] = acres_null['Operations']*395
acres_null

C:\Users\gdsak\AppData\Local\Temp/ipykernel_15280/3336397115.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acres_null['Acres_operated'] = acres_null['Operations']*395


,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
136,CENSUS,2017,FLORIDA,CLAY,12019,120192017,71000.0,96.0,142595,361,255000.0,139.0
168,CENSUS,2017,FLORIDA,FRANKLIN,12037,120372017,2000.0,3.0,5925,15,NaN,4.0
170,CENSUS,2017,FLORIDA,GULF,12045,120452017,8000.0,12.0,18170,46,NaN,14.0
1474,CENSUS,2017,WISCONSIN,MENOMINEE,55078,550782017,NaN,1.0,1185,3,NaN,1.0
1493,CENSUS,2017,WISCONSIN,MILWAUKEE,55079,550792017,128000.0,46.0,33970,86,326000.0,41.0
1607,CENSUS,2012,CALIFORNIA,DEL NORTE,6015,60152012,NaN,22.0,47795,121,545000.0,38.0
1627,CENSUS,2012,CALIFORNIA,ALPINE,6003,60032012,NaN,1.0,1185,3,NaN,1.0
2593,CENSUS,2012,NORTH CAROLINA,DARE,37055,370552012,81000.0,4.0,3555,9,215000.0,6.0
2664,CENSUS,2012,NORTH CAROLINA,SWAIN,37173,371732012,NaN,53.0,37130,94,64000.0,54.0
4630,CENSUS,2002,CALIFORNIA,SAN FRANCISCO,6075,60752002,NaN,6.0,3160,8,5000.0,8.0


In [81]:
#replacing the nulls
coa.loc[coa['Acres_operated'].isnull(), 'Acres_operated'] = acres_null

In [82]:
#checking out the column
coa.isnull().sum()

Program                   0
Year                      0
State                     0
County                    0
FIPS                      0
FIPSYear                  0
Chem_expense_dollars    109
Chem_operations           0
Acres_operated            0
Operations                0
Fert_expense_dollars     62
Fert_operations           0
dtype: int64

In [83]:
fert_ex_null = coa[coa['Fert_expense_dollars'].isnull()] 

In [84]:
fert_ex_null

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
92,CENSUS,2017,CALIFORNIA,HUMBOLDT,6023,60232017,674000.0,123.0,621043.0,849,NaN,253.0
111,CENSUS,2017,CALIFORNIA,ALPINE,6003,60032017,NaN,2.0,3173.0,6,NaN,1.0
112,CENSUS,2017,CALIFORNIA,AMADOR,6005,60052017,1322000.0,227.0,181492.0,482,NaN,182.0
168,CENSUS,2017,FLORIDA,FRANKLIN,12037,120372017,2000.0,3.0,5925.0,15,NaN,4.0
170,CENSUS,2017,FLORIDA,GULF,12045,120452017,8000.0,12.0,18170.0,46,NaN,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5935,CENSUS,2002,TEXAS,PRESIDIO,48377,483772002,NaN,18.0,1503639.0,123,NaN,29.0
5937,CENSUS,2002,TEXAS,TERRELL,48443,484432002,NaN,8.0,1413092.0,76,NaN,3.0
5939,CENSUS,2002,TEXAS,WINKLER,48495,484952002,1000.0,7.0,491718.0,44,NaN,10.0
6011,CENSUS,2002,WISCONSIN,IRON,55051,550512002,25000.0,13.0,12741.0,62,NaN,28.0


In [85]:
fert_ex_null['Fert_expense_dollars'] = fert_ex_null['Fert_operations']*21400
fert_ex_null

C:\Users\gdsak\AppData\Local\Temp/ipykernel_15280/3545088068.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fert_ex_null['Fert_expense_dollars'] = fert_ex_null['Fert_operations']*21400


,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
92,CENSUS,2017,CALIFORNIA,HUMBOLDT,6023,60232017,674000.0,123.0,621043.0,849,5414200.0,253.0
111,CENSUS,2017,CALIFORNIA,ALPINE,6003,60032017,NaN,2.0,3173.0,6,21400.0,1.0
112,CENSUS,2017,CALIFORNIA,AMADOR,6005,60052017,1322000.0,227.0,181492.0,482,3894800.0,182.0
168,CENSUS,2017,FLORIDA,FRANKLIN,12037,120372017,2000.0,3.0,5925.0,15,85600.0,4.0
170,CENSUS,2017,FLORIDA,GULF,12045,120452017,8000.0,12.0,18170.0,46,299600.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5935,CENSUS,2002,TEXAS,PRESIDIO,48377,483772002,NaN,18.0,1503639.0,123,620600.0,29.0
5937,CENSUS,2002,TEXAS,TERRELL,48443,484432002,NaN,8.0,1413092.0,76,64200.0,3.0
5939,CENSUS,2002,TEXAS,WINKLER,48495,484952002,1000.0,7.0,491718.0,44,214000.0,10.0
6011,CENSUS,2002,WISCONSIN,IRON,55051,550512002,25000.0,13.0,12741.0,62,599200.0,28.0


In [86]:
coa.loc[coa['Fert_expense_dollars'].isnull(), 'Fert_expense_dollars'] = fert_ex_null

In [87]:
coa.isnull().sum()

Program                   0
Year                      0
State                     0
County                    0
FIPS                      0
FIPSYear                  0
Chem_expense_dollars    109
Chem_operations           0
Acres_operated            0
Operations                0
Fert_expense_dollars      0
Fert_operations           0
dtype: int64

In [88]:
chem_ex_null = coa[coa['Chem_expense_dollars'].isnull()] 

In [89]:
chem_ex_null

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
91,CENSUS,2017,CALIFORNIA,DEL NORTE,6015,60152017,NaN,23.0,19770.0,90,548000.0,23.0
111,CENSUS,2017,CALIFORNIA,ALPINE,6003,60032017,NaN,2.0,3173.0,6,21400.0,1.0
121,CENSUS,2017,CALIFORNIA,TUOLUMNE,6109,61092017,NaN,103.0,122539.0,417,739000.0,77.0
243,CENSUS,2017,GEORGIA,COBB,13067,130672017,NaN,40.0,2524.0,116,80000.0,49.0
245,CENSUS,2017,GEORGIA,DE KALB,13089,130892017,NaN,5.0,468.0,34,44000.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5933,CENSUS,2002,TEXAS,LOVING,48301,483012002,NaN,1.0,515192.0,14,0.0,0.0
5935,CENSUS,2002,TEXAS,PRESIDIO,48377,483772002,NaN,18.0,1503639.0,123,620600.0,29.0
5937,CENSUS,2002,TEXAS,TERRELL,48443,484432002,NaN,8.0,1413092.0,76,64200.0,3.0
5938,CENSUS,2002,TEXAS,WARD,48475,484752002,NaN,7.0,465639.0,86,17000.0,14.0


In [90]:
chem_ex_null['Chem_expense_dollars'] = chem_ex_null['Chem_operations']*16242
chem_ex_null

C:\Users\gdsak\AppData\Local\Temp/ipykernel_15280/45716824.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chem_ex_null['Chem_expense_dollars'] = chem_ex_null['Chem_operations']*16242


,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
91,CENSUS,2017,CALIFORNIA,DEL NORTE,6015,60152017,373566.0,23.0,19770.0,90,548000.0,23.0
111,CENSUS,2017,CALIFORNIA,ALPINE,6003,60032017,32484.0,2.0,3173.0,6,21400.0,1.0
121,CENSUS,2017,CALIFORNIA,TUOLUMNE,6109,61092017,1672926.0,103.0,122539.0,417,739000.0,77.0
243,CENSUS,2017,GEORGIA,COBB,13067,130672017,649680.0,40.0,2524.0,116,80000.0,49.0
245,CENSUS,2017,GEORGIA,DE KALB,13089,130892017,81210.0,5.0,468.0,34,44000.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5933,CENSUS,2002,TEXAS,LOVING,48301,483012002,16242.0,1.0,515192.0,14,0.0,0.0
5935,CENSUS,2002,TEXAS,PRESIDIO,48377,483772002,292356.0,18.0,1503639.0,123,620600.0,29.0
5937,CENSUS,2002,TEXAS,TERRELL,48443,484432002,129936.0,8.0,1413092.0,76,64200.0,3.0
5938,CENSUS,2002,TEXAS,WARD,48475,484752002,113694.0,7.0,465639.0,86,17000.0,14.0


In [91]:
coa.loc[coa['Chem_expense_dollars'].isnull(), 'Chem_expense_dollars'] = chem_ex_null

In [92]:
coa.isnull().sum()

Program                 0
Year                    0
State                   0
County                  0
FIPS                    0
FIPSYear                0
Chem_expense_dollars    0
Chem_operations         0
Acres_operated          0
Operations              0
Fert_expense_dollars    0
Fert_operations         0
dtype: int64

All of our NaN values have been filled with reasonable substitutions!

All in all, the data sets have been *very* clean. The weather and drought dataframes had few (if any) null values to replace. Much of the work has been on the formatting side of things. Let's check out our USDA data with the describe() function and see how are data is distributed after our cleaning.

In [93]:
coa.describe()

,Year,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
count,6064.000000,6064.000000,6.064000e+03,6.064000e+03,6064.000000,6.064000e+03,6064.000000,6.064000e+03,6064.000000
mean,2009.500000,27448.094327,2.744830e+08,5.818932e+06,357.027540,2.948704e+05,746.732355,8.648256e+06,403.698219
std,5.590631,15091.916231,1.509192e+08,1.352560e+07,325.549621,2.774284e+05,604.412159,1.316485e+07,348.915566
min,2002.000000,5001.000000,5.001200e+07,0.000000e+00,1.000000,7.000000e+00,3.000000,0.000000e+00,0.000000
25%,2005.750000,17040.500000,1.704070e+08,6.327500e+05,144.000000,1.081680e+05,348.000000,1.363000e+06,169.000000
50%,2009.500000,27002.000000,2.700220e+08,2.739000e+06,285.000000,2.471970e+05,614.000000,4.718000e+06,333.000000
75%,2013.250000,37171.500000,3.717170e+08,6.993750e+06,484.250000,4.063318e+05,966.000000,1.155725e+07,548.000000
max,2017.000000,55141.000000,5.514120e+08,3.547430e+08,4935.000000,3.777285e+06,6687.000000,2.698370e+08,5184.000000


In [94]:
#careful with the weather data--you'll need to fix the FIPS stuff because they
#used a different state code (MN is 21 instead of 27)
#see the readme file here: https://www.ncei.noaa.gov/pub/data/cirs/climdiv/county-readme.txt

## Target Variable - In Drought vs Not In Drought
---

Next, we can read in the drought data and use the drought columns to create a total square miles column ('total_Sq_Mi') and an "in drought" column, our target variable. We are defining "in drought" as having over half the county's square mileage in a state of drought or abnormal dryness.

We'll create our "in_drought" column using the total square mile column and the np.where function to calculate and set binary values. 1 will indicate that the county has 50% or more of its area in a drought condition and is *in drought* for the corresponding week. 0 will indicate that the county has less than half of it's area in a drought condition and that the county, for that week, is "not in drought"

In [95]:
drought = pd.read_csv('data/drought.csv')

In [96]:
drought

,Date,FIPS,County,State,Sq_Mi_No_Drought,Sq_Mi_Abnormally_Dry,Sq_Mi_Moderate_Drought,Sq_Mi_Severe_Drought,Sq_Mi_Extreme_Drought,Sq_Mi_Exceptional_Drought,FIPSYearMonth
0,2021-12-28,1033,Colbert County,AL,371.40,253.63,0.0,0.0,0.0,0.0,1033202112
1,2021-12-21,1033,Colbert County,AL,592.73,32.30,0.0,0.0,0.0,0.0,1033202112
2,2021-12-14,1033,Colbert County,AL,625.03,0.00,0.0,0.0,0.0,0.0,1033202112
3,2021-12-07,1033,Colbert County,AL,625.03,0.00,0.0,0.0,0.0,0.0,1033202112
4,2021-11-30,1033,Colbert County,AL,625.03,0.00,0.0,0.0,0.0,0.0,1033202111
...,...,...,...,...,...,...,...,...,...,...,...
3278155,2002-01-29,53029,Island County,WA,213.97,0.00,0.0,0.0,0.0,0.0,53029200201
3278156,2002-01-22,53029,Island County,WA,213.97,0.00,0.0,0.0,0.0,0.0,53029200201
3278157,2002-01-15,53029,Island County,WA,213.97,0.00,0.0,0.0,0.0,0.0,53029200201
3278158,2002-01-08,53029,Island County,WA,213.97,0.00,0.0,0.0,0.0,0.0,53029200201


In [97]:
#step 1 of setting up the Y/Target variable
#combining the square miles for all levels of drought to get the total sq miles for each county
drought['total_Sq_Mi'] = drought['Sq_Mi_No_Drought'] + drought['Sq_Mi_Abnormally_Dry'] + drought['Sq_Mi_Moderate_Drought'] + drought['Sq_Mi_Severe_Drought'] + drought['Sq_Mi_Extreme_Drought'] + drought['Sq_Mi_Exceptional_Drought']
drought

,Date,FIPS,County,State,Sq_Mi_No_Drought,Sq_Mi_Abnormally_Dry,Sq_Mi_Moderate_Drought,Sq_Mi_Severe_Drought,Sq_Mi_Extreme_Drought,Sq_Mi_Exceptional_Drought,FIPSYearMonth,total_Sq_Mi
0,2021-12-28,1033,Colbert County,AL,371.40,253.63,0.0,0.0,0.0,0.0,1033202112,625.03
1,2021-12-21,1033,Colbert County,AL,592.73,32.30,0.0,0.0,0.0,0.0,1033202112,625.03
2,2021-12-14,1033,Colbert County,AL,625.03,0.00,0.0,0.0,0.0,0.0,1033202112,625.03
3,2021-12-07,1033,Colbert County,AL,625.03,0.00,0.0,0.0,0.0,0.0,1033202112,625.03
4,2021-11-30,1033,Colbert County,AL,625.03,0.00,0.0,0.0,0.0,0.0,1033202111,625.03
...,...,...,...,...,...,...,...,...,...,...,...,...
3278155,2002-01-29,53029,Island County,WA,213.97,0.00,0.0,0.0,0.0,0.0,53029200201,213.97
3278156,2002-01-22,53029,Island County,WA,213.97,0.00,0.0,0.0,0.0,0.0,53029200201,213.97
3278157,2002-01-15,53029,Island County,WA,213.97,0.00,0.0,0.0,0.0,0.0,53029200201,213.97
3278158,2002-01-08,53029,Island County,WA,213.97,0.00,0.0,0.0,0.0,0.0,53029200201,213.97


In [98]:
#checking my new column
drought.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3278160 entries, 0 to 3278159
Data columns (total 12 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Date                       object 
 1   FIPS                       int64  
 2   County                     object 
 3   State                      object 
 4   Sq_Mi_No_Drought           float64
 5   Sq_Mi_Abnormally_Dry       float64
 6   Sq_Mi_Moderate_Drought     float64
 7   Sq_Mi_Severe_Drought       float64
 8   Sq_Mi_Extreme_Drought      float64
 9   Sq_Mi_Exceptional_Drought  float64
 10  FIPSYearMonth              int64  
 11  total_Sq_Mi                float64
dtypes: float64(7), int64(2), object(3)
memory usage: 300.1+ MB


In [99]:
#calculating a new binary column for our Target variable, "in_drought" 
drought['in_drought'] = np.where(drought['Sq_Mi_No_Drought'] < (drought['total_Sq_Mi']/2), 1, 0)
drought

,Date,FIPS,County,State,Sq_Mi_No_Drought,Sq_Mi_Abnormally_Dry,Sq_Mi_Moderate_Drought,Sq_Mi_Severe_Drought,Sq_Mi_Extreme_Drought,Sq_Mi_Exceptional_Drought,FIPSYearMonth,total_Sq_Mi,in_drought
0,2021-12-28,1033,Colbert County,AL,371.40,253.63,0.0,0.0,0.0,0.0,1033202112,625.03,0
1,2021-12-21,1033,Colbert County,AL,592.73,32.30,0.0,0.0,0.0,0.0,1033202112,625.03,0
2,2021-12-14,1033,Colbert County,AL,625.03,0.00,0.0,0.0,0.0,0.0,1033202112,625.03,0
3,2021-12-07,1033,Colbert County,AL,625.03,0.00,0.0,0.0,0.0,0.0,1033202112,625.03,0
4,2021-11-30,1033,Colbert County,AL,625.03,0.00,0.0,0.0,0.0,0.0,1033202111,625.03,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3278155,2002-01-29,53029,Island County,WA,213.97,0.00,0.0,0.0,0.0,0.0,53029200201,213.97,0
3278156,2002-01-22,53029,Island County,WA,213.97,0.00,0.0,0.0,0.0,0.0,53029200201,213.97,0
3278157,2002-01-15,53029,Island County,WA,213.97,0.00,0.0,0.0,0.0,0.0,53029200201,213.97,0
3278158,2002-01-08,53029,Island County,WA,213.97,0.00,0.0,0.0,0.0,0.0,53029200201,213.97,0


In [100]:
drought[drought['in_drought']==1]

,Date,FIPS,County,State,Sq_Mi_No_Drought,Sq_Mi_Abnormally_Dry,Sq_Mi_Moderate_Drought,Sq_Mi_Severe_Drought,Sq_Mi_Extreme_Drought,Sq_Mi_Exceptional_Drought,FIPSYearMonth,total_Sq_Mi,in_drought
79,2020-06-23,1033,Colbert County,AL,167.27,457.76,0.00,0.0,0.0,0.0,1033202006,625.03,1
80,2020-06-16,1033,Colbert County,AL,0.00,625.03,0.00,0.0,0.0,0.0,1033202006,625.03,1
117,2019-10-01,1033,Colbert County,AL,0.00,512.16,112.87,0.0,0.0,0.0,1033201910,625.03,1
118,2019-09-24,1033,Colbert County,AL,228.80,396.23,0.00,0.0,0.0,0.0,1033201909,625.03,1
130,2019-07-02,1033,Colbert County,AL,26.83,598.20,0.00,0.0,0.0,0.0,1033201907,625.03,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3278111,2002-12-03,53029,Island County,WA,0.00,213.97,0.00,0.0,0.0,0.0,53029200212,213.97,1
3278112,2002-11-26,53029,Island County,WA,0.00,213.97,0.00,0.0,0.0,0.0,53029200211,213.97,1
3278113,2002-11-19,53029,Island County,WA,0.00,213.97,0.00,0.0,0.0,0.0,53029200211,213.97,1
3278114,2002-11-12,53029,Island County,WA,0.00,213.97,0.00,0.0,0.0,0.0,53029200211,213.97,1


In [101]:
print(f'There are {drought["in_drought"].value_counts()[0]} instances where a county is "Not In Drought" for a week\n\
and {drought["in_drought"].value_counts()[1]} instances where a county is "In Drought" for a week.\n\
This amounts to a \
{round(drought["in_drought"].value_counts()[1]/(drought["in_drought"].value_counts()[0]+drought["in_drought"].value_counts()[1])*100, 2)}% chance \
a county in the United States is in drought any given week.')

There are 2071449 instances where a county is "Not In Drought" for a week
and 1206711 instances where a county is "In Drought" for a week.
This amounts to a 36.81% chance a county in the United States is in drought any given week.


We have a modest imbalance in our positive, "in drought" class and our negative "not in drought" class. We will be narrowing down our data to the 15 states with the most agricultural production, the "Farmland" counties. We can recheck this balance of classes on our final dataset.

If we are lucky, this imbalance will even out a bit once we narrow the scope of the data. If that happens, it will make our modeling more straightforward AND it will hint at a higher rate of drought where farming is more common.

Before cutting out a subset to move on with, we can write it to a new csv file. This way, we can perform some analysis on the nationwide drought data so we can compare the Farmland to the US totals.

In [102]:
#making a copy of the nationwide drought data for some analysis
drought.to_csv('drought_full.csv', index=False)

In order to filter down to the 15 states we will set the 'State' column equal to a list of the states with the highest agricultural production. This can fluctuate a bit year to year, but historically the states have been:

AR - Arkansas
CA - California
FL - Florida
GA - Georgia
IL - Illinios
IN - Indiana
IA - Iowa
KS - Kansas
MN - Minnesota
MO - Missouri
NE - Nebraska
NC - North Carolina
TX - Texas
WA - Washington
WI - Wisconsin

These are the states that the USDA has included in their yearly **Survey of Agriculture** in recent memory. The survey has state-level data and could help us with some analysis later on. Because the data we need is at the county level, we will need to dial in the drought dataset to only use years that we have full, ***Census* of Agriculture** data. This is the years 2002, 2007, 2012, and 2017.

Once we have drought down to the subset of 15 states, we can filter out years so we are left with the 4 years that have complete data across our 3 datasets.

In [103]:
drought['State'].unique()

array(['AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'ID', 'IL',
       'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'DC', 'MA', 'RI', 'MI',
       'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC',
       'ND', 'OH', 'OK', 'OR', 'PA', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT',
       'VA', 'WA', 'WV', 'WI', 'WY', 'AK', 'HI'], dtype=object)

In [104]:
#setting a list up with the 15 farmland states
farmstates= ['AR', 'CA', 'FL', 'GA', 'IL', 'IN', 'IA', 'KS', 'MN', 'MO', 'NE', 'NC', 'TX', 'WA', 'WI']

In [105]:
#resetting the dataframe, should cut down to ~1.5 million rows
drought = drought.loc[drought['State'].isin(farmstates)]
drought

,Date,FIPS,County,State,Sq_Mi_No_Drought,Sq_Mi_Abnormally_Dry,Sq_Mi_Moderate_Drought,Sq_Mi_Severe_Drought,Sq_Mi_Extreme_Drought,Sq_Mi_Exceptional_Drought,FIPSYearMonth,total_Sq_Mi,in_drought
42886,2021-12-28,5007,Benton County,AR,873.97,0.0,0.0,0.0,0.0,0.0,5007202112,873.97,0
42887,2021-12-21,5007,Benton County,AR,873.97,0.0,0.0,0.0,0.0,0.0,5007202112,873.97,0
42888,2021-12-14,5007,Benton County,AR,703.57,170.4,0.0,0.0,0.0,0.0,5007202112,873.97,0
42889,2021-12-07,5007,Benton County,AR,703.57,170.4,0.0,0.0,0.0,0.0,5007202112,873.97,0
42890,2021-11-30,5007,Benton County,AR,703.57,170.4,0.0,0.0,0.0,0.0,5007202111,873.97,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3278155,2002-01-29,53029,Island County,WA,213.97,0.0,0.0,0.0,0.0,0.0,53029200201,213.97,0
3278156,2002-01-22,53029,Island County,WA,213.97,0.0,0.0,0.0,0.0,0.0,53029200201,213.97,0
3278157,2002-01-15,53029,Island County,WA,213.97,0.0,0.0,0.0,0.0,0.0,53029200201,213.97,0
3278158,2002-01-08,53029,Island County,WA,213.97,0.0,0.0,0.0,0.0,0.0,53029200201,213.97,0


In [106]:
#checking that we have the 15 states we need
drought['State'].unique()

array(['AR', 'CA', 'FL', 'GA', 'IL', 'IN', 'IA', 'KS', 'MN', 'MO', 'NE',
       'NC', 'TX', 'WA', 'WI'], dtype=object)

We will need to filter by years. We can create a 'Year' column by extracting the first four positions of the "date' column and use that to filter. Because we need to add a 'FIPSYear' column to the drought dataframe to merge in our yearly USDA data, we'll keep the Year column around for a bit to so we can set up a FIPSYear column later on.

In [107]:
#checking that the years are correctly extracted
drought['Date'].str.extract('(.{4})')

,0
42886,2021
42887,2021
42888,2021
42889,2021
42890,2021
...,...
3278155,2002
3278156,2002
3278157,2002
3278158,2002


In [108]:
#adding the column and checking the df
drought['Year'] = drought['Date'].str.extract('(.{4})')
drought

C:\Users\gdsak\AppData\Local\Temp/ipykernel_15280/865174138.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drought['Year'] = drought['Date'].str.extract('(.{4})')


,Date,FIPS,County,State,Sq_Mi_No_Drought,Sq_Mi_Abnormally_Dry,Sq_Mi_Moderate_Drought,Sq_Mi_Severe_Drought,Sq_Mi_Extreme_Drought,Sq_Mi_Exceptional_Drought,FIPSYearMonth,total_Sq_Mi,in_drought,Year
42886,2021-12-28,5007,Benton County,AR,873.97,0.0,0.0,0.0,0.0,0.0,5007202112,873.97,0,2021
42887,2021-12-21,5007,Benton County,AR,873.97,0.0,0.0,0.0,0.0,0.0,5007202112,873.97,0,2021
42888,2021-12-14,5007,Benton County,AR,703.57,170.4,0.0,0.0,0.0,0.0,5007202112,873.97,0,2021
42889,2021-12-07,5007,Benton County,AR,703.57,170.4,0.0,0.0,0.0,0.0,5007202112,873.97,0,2021
42890,2021-11-30,5007,Benton County,AR,703.57,170.4,0.0,0.0,0.0,0.0,5007202111,873.97,0,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3278155,2002-01-29,53029,Island County,WA,213.97,0.0,0.0,0.0,0.0,0.0,53029200201,213.97,0,2002
3278156,2002-01-22,53029,Island County,WA,213.97,0.0,0.0,0.0,0.0,0.0,53029200201,213.97,0,2002
3278157,2002-01-15,53029,Island County,WA,213.97,0.0,0.0,0.0,0.0,0.0,53029200201,213.97,0,2002
3278158,2002-01-08,53029,Island County,WA,213.97,0.0,0.0,0.0,0.0,0.0,53029200201,213.97,0,2002


In [109]:
#making sure we don't have nans and checking the data type
drought.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1583748 entries, 42886 to 3278159
Data columns (total 14 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Date                       1583748 non-null  object 
 1   FIPS                       1583748 non-null  int64  
 2   County                     1583748 non-null  object 
 3   State                      1583748 non-null  object 
 4   Sq_Mi_No_Drought           1583748 non-null  float64
 5   Sq_Mi_Abnormally_Dry       1583748 non-null  float64
 6   Sq_Mi_Moderate_Drought     1583748 non-null  float64
 7   Sq_Mi_Severe_Drought       1583748 non-null  float64
 8   Sq_Mi_Extreme_Drought      1583748 non-null  float64
 9   Sq_Mi_Exceptional_Drought  1583748 non-null  float64
 10  FIPSYearMonth              1583748 non-null  int64  
 11  total_Sq_Mi                1583748 non-null  float64
 12  in_drought                 1583748 non-null  int32  
 13  Year    

In [110]:
completeyears = ['2002', '2007', '2012', '2017']

In [111]:
#we should have about 300k rows
drought.loc[drought['Year'].isin(completeyears)]

,Date,FIPS,County,State,Sq_Mi_No_Drought,Sq_Mi_Abnormally_Dry,Sq_Mi_Moderate_Drought,Sq_Mi_Severe_Drought,Sq_Mi_Extreme_Drought,Sq_Mi_Exceptional_Drought,FIPSYearMonth,total_Sq_Mi,in_drought,Year
43095,2017-12-26,5007,Benton County,AR,0.00,222.84,651.13,0.00,0.0,0.0,5007201712,873.97,1,2017
43096,2017-12-19,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,5007201712,873.97,1,2017
43097,2017-12-12,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,5007201712,873.97,1,2017
43098,2017-12-05,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,5007201712,873.97,1,2017
43099,2017-11-28,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,5007201711,873.97,1,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3278155,2002-01-29,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002
3278156,2002-01-22,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002
3278157,2002-01-15,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002
3278158,2002-01-08,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002


In [112]:
#resetting the dataframe
drought = drought.loc[drought['Year'].isin(completeyears)]
drought.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 317053 entries, 43095 to 3278159
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Date                       317053 non-null  object 
 1   FIPS                       317053 non-null  int64  
 2   County                     317053 non-null  object 
 3   State                      317053 non-null  object 
 4   Sq_Mi_No_Drought           317053 non-null  float64
 5   Sq_Mi_Abnormally_Dry       317053 non-null  float64
 6   Sq_Mi_Moderate_Drought     317053 non-null  float64
 7   Sq_Mi_Severe_Drought       317053 non-null  float64
 8   Sq_Mi_Extreme_Drought      317053 non-null  float64
 9   Sq_Mi_Exceptional_Drought  317053 non-null  float64
 10  FIPSYearMonth              317053 non-null  int64  
 11  total_Sq_Mi                317053 non-null  float64
 12  in_drought                 317053 non-null  int32  
 13  Year                    

In [113]:
drought.reset_index(drop=True, inplace=True)
drought

,Date,FIPS,County,State,Sq_Mi_No_Drought,Sq_Mi_Abnormally_Dry,Sq_Mi_Moderate_Drought,Sq_Mi_Severe_Drought,Sq_Mi_Extreme_Drought,Sq_Mi_Exceptional_Drought,FIPSYearMonth,total_Sq_Mi,in_drought,Year
0,2017-12-26,5007,Benton County,AR,0.00,222.84,651.13,0.00,0.0,0.0,5007201712,873.97,1,2017
1,2017-12-19,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,5007201712,873.97,1,2017
2,2017-12-12,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,5007201712,873.97,1,2017
3,2017-12-05,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,5007201712,873.97,1,2017
4,2017-11-28,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,5007201711,873.97,1,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317048,2002-01-29,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002
317049,2002-01-22,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002
317050,2002-01-15,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002
317051,2002-01-08,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002


Our drought data has been filtered down to the 15 Farmland states for the 4 years in which we have complete data! 
We can bring in the last dataset, 'weather', and double check it before performing some similar filtering to get the years and states down to the core dataset we'll be using.

In [114]:
weather = pd.read_csv('data/weather.csv')
weather

,FIPSYearMonth,FIPS,Year,Month,precip,mintemp,maxtemp,avgtemp
0,1001200201,1001,2002,1,4.70,36.1,59.2,47.6
1,1001200301,1001,2003,1,2.62,29.2,52.6,41.0
2,1001200401,1001,2004,1,2.60,33.8,56.2,45.0
3,1001200501,1001,2005,1,3.08,38.1,61.6,49.9
4,1001200601,1001,2006,1,5.69,40.1,63.8,52.0
...,...,...,...,...,...,...,...,...
752875,50290201712,50290,2017,12,0.82,8.0,20.4,14.2
752876,50290201812,50290,2018,12,0.88,-3.7,9.1,2.7
752877,50290201912,50290,2019,12,0.58,-11.4,1.6,-4.9
752878,50290202012,50290,2020,12,0.52,-3.2,9.4,3.1


In [115]:
#checking for nans and the datatypes
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752880 entries, 0 to 752879
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   FIPSYearMonth  752880 non-null  int64  
 1   FIPS           752880 non-null  int64  
 2   Year           752880 non-null  int64  
 3   Month          752880 non-null  int64  
 4   precip         752880 non-null  float64
 5   mintemp        752880 non-null  float64
 6   maxtemp        752880 non-null  float64
 7   avgtemp        752880 non-null  float64
dtypes: float64(4), int64(4)
memory usage: 46.0 MB


Let's check for the author's home county, Hennepin County, Minnesota: **FIPS 27053**

In [116]:
weather[weather['FIPS']==27053]

,FIPSYearMonth,FIPS,Year,Month,precip,mintemp,maxtemp,avgtemp


...Well that's not right. 

After a quick review of readme.txt files in the weather data's source, (https://www.ncei.noaa.gov/pub/data/cirs/climdiv/county-readme.txt) it seems that the FIPS numbers don't *quite* follow the FIPS numbers we have in our other two datasets. The weather data's "FIPS" data (imposter!) follows the below numbering conventions:

STATE CODE TABLE: 
                             Range of values of 01-48.

                             01 Alabama                 28 New Jersey
                             02 Arizona                 29 New Mexico
                             03 Arkansas                30 New York
                             04 California              31 North Carolina
                             05 Colorado                32 North Dakota
                             06 Connecticut             33 Ohio
                             07 Delaware                34 Oklahoma
                             08 Florida                 35 Oregon
                             09 Georgia                 36 Pennsylvania
                             10 Idaho                   37 Rhode Island
                             11 Illinois                38 South Carolina
                             12 Indiana                 39 South Dakota
                             13 Iowa                    40 Tennessee
                             14 Kansas                  41 Texas
                             15 Kentucky                42 Utah
                             16 Louisiana               43 Vermont
                             17 Maine                   44 Virginia
                             18 Maryland                45 Washington
                             19 Massachusetts           46 West Virginia
                             20 Michigan                47 Wisconsin
                             21 Minnesota               48 Wyoming
                             22 Mississippi             50 Alaska
                             23 Missouri   
                             24 Montana   
                             25 Nebraska 
                             26 Nevada  
                             27 New Hampshire
                             
Meanwhile, the USDA (https://www.nrcs.usda.gov/wps/portal/nrcs/detail/?cid=nrcs143_013696) has the following for their FIPS numbers:

- Alabama	AL	01
- Alaska	AK	02
- Arizona	AZ	04
- Arkansas	AR	05
- California	CA	06
- Colorado	CO	08
- Connecticut	CT	09
- Delaware	DE	10
- Florida	FL	12
- Georgia	GA	13
- Hawaii	HI	15
- Idaho	ID	16
- Illinois	IL	17
- Indiana	IN	18
- Iowa	IA	19
- Kansas	KS	20
- Kentucky	KY	21
- Louisiana	LA	22
- Maine	ME	23
- Maryland	MD	24
- Massachusetts	MA	25
- Michigan	MI	26
- Minnesota	MN	27
- Mississippi	MS	28
- Missouri	MO	29
- Montana	MT	30
- Nebraska	NE	31
- Nevada	NV	32
- New Hampshire	NH	33
- New Jersey	NJ	34
- New Mexico	NM	35
- New York	NY	36
- North Carolina	NC	37
- North Dakota	ND	38
- Ohio	OH	39
- Oklahoma	OK	40
- Oregon	OR	41
- Pennsylvania	PA	42
- Rhode Island	RI	44
- South Carolina	SC	45
- South Dakota	SD	46
- Tennessee	TN	47
- Texas	TX	48
- Utah	UT	49
- Vermont	VT	50
- Virginia	VA	51
- Washington	WA	53
- West Virginia	WV	54
- Wisconsin	WI	55
- Wyoming	WY	56


Good thing we only need to fix the FIPS data for 15 states!

So Hennepin county, MN *should* be FIPS **21053** in the weather data. Let's try again and check if the weather data is reasonable for Hennepin County.

In [117]:
#calling in the top 20 rows should give us january data
#it should be cold. Below freezing (32 d fahrenheit) most of the years.
weather[weather['FIPS']==21053].head(20)

,FIPSYearMonth,FIPS,Year,Month,precip,mintemp,maxtemp,avgtemp
26120,21053200201,21053,2002,1,0.40,15.2,30.6,22.9
26121,21053200301,21053,2003,1,0.24,5.2,22.6,13.9
26122,21053200401,21053,2004,1,0.40,2.1,18.2,10.2
26123,21053200501,21053,2005,1,1.05,6.4,22.1,14.3
26124,21053200601,21053,2006,1,0.56,21.0,33.7,27.3
26125,21053200701,21053,2007,1,0.42,10.4,25.4,17.9
26126,21053200801,21053,2008,1,0.10,3.4,21.0,12.2
26127,21053200901,21053,2009,1,0.41,-2.6,16.4,6.9
26128,21053201001,21053,2010,1,0.52,4.0,19.1,11.5
26129,21053201101,21053,2011,1,0.84,2.6,17.5,10.0


Yeah, thats more like it. Average temperaturess that are consistently below freezing in January. Home sweet home.

First we can separate out the state and county portions of the FIPS numbers in 'weather'. Then we can filter down to the (wrong) FIPS for the 15 states we need. Then we will reset the *state* FIPS, drop the old FIPS column, and pull the corrected state and county numbers together again to make a new, correct FIPS column.

- 'AR' - 03 becomes 05
- 'CA' - 04 becomes 06
- 'FL' - 08 becomes 12
- 'GA' - 09 becomes 13
- 'IL' - 11 becomes 17
- 'IN' - 12 becomes 18
- 'IA' - 13 becomes 19
- 'KS' - 14 becomes 20
- 'MN' - 21 becomes 27
- 'MO' - 23 becomes 29
- 'NE' - 25 becomes 31
- 'NC' - 32 becomes 37
- 'TX' - 41 becomes 48
- 'WA' - 45 becomes 53
- 'WI' - 47 becomes 55

In [118]:
weather['FIPS'] = weather['FIPS'].astype(str)
weather['FIPS'] = weather['FIPS'].str.zfill(5)
weather[['state_FIPS', 'cty_FIPS']] = weather['FIPS'].str.extract('(.{2})(.{3})')
weather

,FIPSYearMonth,FIPS,Year,Month,precip,mintemp,maxtemp,avgtemp,state_FIPS,cty_FIPS
0,1001200201,01001,2002,1,4.70,36.1,59.2,47.6,01,001
1,1001200301,01001,2003,1,2.62,29.2,52.6,41.0,01,001
2,1001200401,01001,2004,1,2.60,33.8,56.2,45.0,01,001
3,1001200501,01001,2005,1,3.08,38.1,61.6,49.9,01,001
4,1001200601,01001,2006,1,5.69,40.1,63.8,52.0,01,001
...,...,...,...,...,...,...,...,...,...,...
752875,50290201712,50290,2017,12,0.82,8.0,20.4,14.2,50,290
752876,50290201812,50290,2018,12,0.88,-3.7,9.1,2.7,50,290
752877,50290201912,50290,2019,12,0.58,-11.4,1.6,-4.9,50,290
752878,50290202012,50290,2020,12,0.52,-3.2,9.4,3.1,50,290


In [119]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752880 entries, 0 to 752879
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   FIPSYearMonth  752880 non-null  int64  
 1   FIPS           752880 non-null  object 
 2   Year           752880 non-null  int64  
 3   Month          752880 non-null  int64  
 4   precip         752880 non-null  float64
 5   mintemp        752880 non-null  float64
 6   maxtemp        752880 non-null  float64
 7   avgtemp        752880 non-null  float64
 8   state_FIPS     752880 non-null  object 
 9   cty_FIPS       752880 non-null  object 
dtypes: float64(4), int64(3), object(3)
memory usage: 57.4+ MB


In [120]:
farmstatesWRONGfips = ['03', '04', '08', '09', '11', '12', '13', '14', '21', '23', '25', '32', '41', '45', '47']

In [121]:
#about half of the data should be left, over 300k rows
weather.loc[weather['state_FIPS'].isin(farmstatesWRONGfips)]

,FIPSYearMonth,FIPS,Year,Month,precip,mintemp,maxtemp,avgtemp,state_FIPS,cty_FIPS
1640,3001200201,03001,2002,1,3.63,35.8,54.2,45.0,03,001
1641,3001200301,03001,2003,1,0.44,28.9,47.1,38.0,03,001
1642,3001200401,03001,2004,1,5.08,34.3,52.2,43.3,03,001
1643,3001200501,03001,2005,1,4.80,38.7,54.0,46.4,03,001
1644,3001200601,03001,2006,1,6.62,38.7,58.8,48.7,03,001
...,...,...,...,...,...,...,...,...,...,...
751815,47141201712,47141,2017,12,1.09,8.8,24.5,16.6,47,141
751816,47141201812,47141,2018,12,1.41,17.6,31.6,24.6,47,141
751817,47141201912,47141,2019,12,2.69,15.0,30.7,22.8,47,141
751818,47141202012,47141,2020,12,0.29,16.6,32.9,24.7,47,141


In [122]:
weather = weather.loc[weather['state_FIPS'].isin(farmstatesWRONGfips)]
weather

,FIPSYearMonth,FIPS,Year,Month,precip,mintemp,maxtemp,avgtemp,state_FIPS,cty_FIPS
1640,3001200201,03001,2002,1,3.63,35.8,54.2,45.0,03,001
1641,3001200301,03001,2003,1,0.44,28.9,47.1,38.0,03,001
1642,3001200401,03001,2004,1,5.08,34.3,52.2,43.3,03,001
1643,3001200501,03001,2005,1,4.80,38.7,54.0,46.4,03,001
1644,3001200601,03001,2006,1,6.62,38.7,58.8,48.7,03,001
...,...,...,...,...,...,...,...,...,...,...
751815,47141201712,47141,2017,12,1.09,8.8,24.5,16.6,47,141
751816,47141201812,47141,2018,12,1.41,17.6,31.6,24.6,47,141
751817,47141201912,47141,2019,12,2.69,15.0,30.7,22.8,47,141
751818,47141202012,47141,2020,12,0.29,16.6,32.9,24.7,47,141


In [123]:
#setting up the RIGHT fips values
farmstatesRIGHTfips = ['05', '06', '12', '13', '17', '18', '19', '20', '27', '29', '31', '37', '48', '53', '55']

In [124]:
#checking the zip function 
farmfipsswap = zip(farmstatesWRONGfips, farmstatesRIGHTfips)
list(farmfipsswap)

[('03', '05'),
 ('04', '06'),
 ('08', '12'),
 ('09', '13'),
 ('11', '17'),
 ('12', '18'),
 ('13', '19'),
 ('14', '20'),
 ('21', '27'),
 ('23', '29'),
 ('25', '31'),
 ('32', '37'),
 ('41', '48'),
 ('45', '53'),
 ('47', '55')]

In [125]:
#time for a trusty for loop
#make a new column called stateFIPS so we can eyeball the first and last 5 values
for i, j in zip(farmstatesWRONGfips, farmstatesRIGHTfips):
    weather.loc[weather['state_FIPS']== i, 'stateFIPS'] = j

weather

C:\Users\gdsak\anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\gdsak\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,FIPSYearMonth,FIPS,Year,Month,precip,mintemp,maxtemp,avgtemp,state_FIPS,cty_FIPS,stateFIPS
1640,3001200201,03001,2002,1,3.63,35.8,54.2,45.0,03,001,05
1641,3001200301,03001,2003,1,0.44,28.9,47.1,38.0,03,001,05
1642,3001200401,03001,2004,1,5.08,34.3,52.2,43.3,03,001,05
1643,3001200501,03001,2005,1,4.80,38.7,54.0,46.4,03,001,05
1644,3001200601,03001,2006,1,6.62,38.7,58.8,48.7,03,001,05
...,...,...,...,...,...,...,...,...,...,...,...
751815,47141201712,47141,2017,12,1.09,8.8,24.5,16.6,47,141,55
751816,47141201812,47141,2018,12,1.41,17.6,31.6,24.6,47,141,55
751817,47141201912,47141,2019,12,2.69,15.0,30.7,22.8,47,141,55
751818,47141202012,47141,2020,12,0.29,16.6,32.9,24.7,47,141,55


In [126]:
weather = weather.drop(columns=['FIPS'])
weather

,FIPSYearMonth,Year,Month,precip,mintemp,maxtemp,avgtemp,state_FIPS,cty_FIPS,stateFIPS
1640,3001200201,2002,1,3.63,35.8,54.2,45.0,03,001,05
1641,3001200301,2003,1,0.44,28.9,47.1,38.0,03,001,05
1642,3001200401,2004,1,5.08,34.3,52.2,43.3,03,001,05
1643,3001200501,2005,1,4.80,38.7,54.0,46.4,03,001,05
1644,3001200601,2006,1,6.62,38.7,58.8,48.7,03,001,05
...,...,...,...,...,...,...,...,...,...,...
751815,47141201712,2017,12,1.09,8.8,24.5,16.6,47,141,55
751816,47141201812,2018,12,1.41,17.6,31.6,24.6,47,141,55
751817,47141201912,2019,12,2.69,15.0,30.7,22.8,47,141,55
751818,47141202012,2020,12,0.29,16.6,32.9,24.7,47,141,55


In [127]:
weather['FIPS'] = weather['stateFIPS'] + weather['cty_FIPS']
weather

,FIPSYearMonth,Year,Month,precip,mintemp,maxtemp,avgtemp,state_FIPS,cty_FIPS,stateFIPS,FIPS
1640,3001200201,2002,1,3.63,35.8,54.2,45.0,03,001,05,05001
1641,3001200301,2003,1,0.44,28.9,47.1,38.0,03,001,05,05001
1642,3001200401,2004,1,5.08,34.3,52.2,43.3,03,001,05,05001
1643,3001200501,2005,1,4.80,38.7,54.0,46.4,03,001,05,05001
1644,3001200601,2006,1,6.62,38.7,58.8,48.7,03,001,05,05001
...,...,...,...,...,...,...,...,...,...,...,...
751815,47141201712,2017,12,1.09,8.8,24.5,16.6,47,141,55,55141
751816,47141201812,2018,12,1.41,17.6,31.6,24.6,47,141,55,55141
751817,47141201912,2019,12,2.69,15.0,30.7,22.8,47,141,55,55141
751818,47141202012,2020,12,0.29,16.6,32.9,24.7,47,141,55,55141


In [128]:
weather = weather.drop(columns=['state_FIPS', 'cty_FIPS', 'stateFIPS'])
weather

,FIPSYearMonth,Year,Month,precip,mintemp,maxtemp,avgtemp,FIPS
1640,3001200201,2002,1,3.63,35.8,54.2,45.0,05001
1641,3001200301,2003,1,0.44,28.9,47.1,38.0,05001
1642,3001200401,2004,1,5.08,34.3,52.2,43.3,05001
1643,3001200501,2005,1,4.80,38.7,54.0,46.4,05001
1644,3001200601,2006,1,6.62,38.7,58.8,48.7,05001
...,...,...,...,...,...,...,...,...
751815,47141201712,2017,12,1.09,8.8,24.5,16.6,55141
751816,47141201812,2018,12,1.41,17.6,31.6,24.6,55141
751817,47141201912,2019,12,2.69,15.0,30.7,22.8,55141
751818,47141202012,2020,12,0.29,16.6,32.9,24.7,55141


In [129]:
#checking for nulls
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 352800 entries, 1640 to 751819
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   FIPSYearMonth  352800 non-null  int64  
 1   Year           352800 non-null  int64  
 2   Month          352800 non-null  int64  
 3   precip         352800 non-null  float64
 4   mintemp        352800 non-null  float64
 5   maxtemp        352800 non-null  float64
 6   avgtemp        352800 non-null  float64
 7   FIPS           352800 non-null  object 
dtypes: float64(4), int64(3), object(1)
memory usage: 24.2+ MB


In [130]:
#setting up a FIPSYear column to make combining the dataframes easier later on
weather['Year'] = weather['Year'].astype(str)
weather['FIPSYear'] = weather['FIPS'] + weather['Year']
weather['Year'] = weather['Year'].astype(int)
weather

,FIPSYearMonth,Year,Month,precip,mintemp,maxtemp,avgtemp,FIPS,FIPSYear
1640,3001200201,2002,1,3.63,35.8,54.2,45.0,05001,050012002
1641,3001200301,2003,1,0.44,28.9,47.1,38.0,05001,050012003
1642,3001200401,2004,1,5.08,34.3,52.2,43.3,05001,050012004
1643,3001200501,2005,1,4.80,38.7,54.0,46.4,05001,050012005
1644,3001200601,2006,1,6.62,38.7,58.8,48.7,05001,050012006
...,...,...,...,...,...,...,...,...,...
751815,47141201712,2017,12,1.09,8.8,24.5,16.6,55141,551412017
751816,47141201812,2018,12,1.41,17.6,31.6,24.6,55141,551412018
751817,47141201912,2019,12,2.69,15.0,30.7,22.8,55141,551412019
751818,47141202012,2020,12,0.29,16.6,32.9,24.7,55141,551412020


In [131]:
#setting up a numeric list of years
completeyearsnum = [2002, 2007, 2012, 2017]

In [132]:
#checking the filtered list of years
#we should have ~70k
weather.loc[weather['Year'].isin(completeyearsnum)]

,FIPSYearMonth,Year,Month,precip,mintemp,maxtemp,avgtemp,FIPS,FIPSYear
1640,3001200201,2002,1,3.63,35.8,54.2,45.0,05001,050012002
1645,3001200701,2007,1,6.33,34.6,49.8,42.2,05001,050012007
1650,3001201201,2012,1,2.07,37.2,57.0,47.1,05001,050012012
1655,3001201701,2017,1,2.92,38.3,55.7,47.0,05001,050012017
1660,3003200201,2002,1,7.24,34.5,57.4,46.0,05003,050032002
...,...,...,...,...,...,...,...,...,...
751795,47139201712,2017,12,0.80,12.6,26.8,19.7,55139,551392017
751800,47141200212,2002,12,0.29,15.4,32.4,23.9,55141,551412002
751805,47141200712,2007,12,2.36,7.9,25.2,16.6,55141,551412007
751810,47141201212,2012,12,1.52,17.3,32.2,24.7,55141,551412012


In [133]:
weather = weather.loc[weather['Year'].isin(completeyearsnum)]
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70560 entries, 1640 to 751815
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   FIPSYearMonth  70560 non-null  int64  
 1   Year           70560 non-null  int32  
 2   Month          70560 non-null  int64  
 3   precip         70560 non-null  float64
 4   mintemp        70560 non-null  float64
 5   maxtemp        70560 non-null  float64
 6   avgtemp        70560 non-null  float64
 7   FIPS           70560 non-null  object 
 8   FIPSYear       70560 non-null  object 
dtypes: float64(4), int32(1), int64(2), object(2)
memory usage: 5.1+ MB


In [134]:
weather.reset_index(drop=True, inplace=True)

In [135]:
drought.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317053 entries, 0 to 317052
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Date                       317053 non-null  object 
 1   FIPS                       317053 non-null  int64  
 2   County                     317053 non-null  object 
 3   State                      317053 non-null  object 
 4   Sq_Mi_No_Drought           317053 non-null  float64
 5   Sq_Mi_Abnormally_Dry       317053 non-null  float64
 6   Sq_Mi_Moderate_Drought     317053 non-null  float64
 7   Sq_Mi_Severe_Drought       317053 non-null  float64
 8   Sq_Mi_Extreme_Drought      317053 non-null  float64
 9   Sq_Mi_Exceptional_Drought  317053 non-null  float64
 10  FIPSYearMonth              317053 non-null  int64  
 11  total_Sq_Mi                317053 non-null  float64
 12  in_drought                 317053 non-null  int32  
 13  Year                       31

With all three dataframes thoroughly vetted and reformatted we can work on combining them. Drought and Weather can be combined first. In order to avoid duplicates, we can drop a few columns from weather prior to merging the dataframes.

Drought and weather have several columns in common: FIPS, Year, and FIPSYearMonth, the column we will use to join them. We can drop FIPS and Year from weather. FIPSYearMonth is still set up with the old, incorrect FIPS numbers, so we will need to drop and recreate that before the merger.

In [136]:
weather = weather.drop(columns=['FIPS', 'Year'])

In [137]:
weather = weather.drop(columns=['FIPSYearMonth'])

In [138]:
#changing the month column to string and filling with leading zeros
weather['Month'] = weather['Month'].astype(str)
weather['Month'] = weather['Month'].str.zfill(2)

#recreating the FIPSYearMonth column
weather['FIPSYearMonth'] = weather['FIPSYear'] + weather['Month']

#changing Month back to int
weather['Month'] = weather['Month'].astype(int)
weather

,Month,precip,mintemp,maxtemp,avgtemp,FIPSYear,FIPSYearMonth
0,1,3.63,35.8,54.2,45.0,050012002,05001200201
1,1,6.33,34.6,49.8,42.2,050012007,05001200701
2,1,2.07,37.2,57.0,47.1,050012012,05001201201
3,1,2.92,38.3,55.7,47.0,050012017,05001201701
4,1,7.24,34.5,57.4,46.0,050032002,05003200201
...,...,...,...,...,...,...,...
70555,12,0.80,12.6,26.8,19.7,551392017,55139201712
70556,12,0.29,15.4,32.4,23.9,551412002,55141200212
70557,12,2.36,7.9,25.2,16.6,551412007,55141200712
70558,12,1.52,17.3,32.2,24.7,551412012,55141201212


In [139]:
#changing the FIPSYearMonth column on drought to a string and filling in leading zeros
drought['FIPSYearMonth'] = drought['FIPSYearMonth'].astype(str)
drought['FIPSYearMonth'] = drought['FIPSYearMonth'].str.zfill(11)
drought

C:\Users\gdsak\AppData\Local\Temp/ipykernel_15280/241469064.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drought['FIPSYearMonth'] = drought['FIPSYearMonth'].astype(str)
C:\Users\gdsak\AppData\Local\Temp/ipykernel_15280/241469064.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drought['FIPSYearMonth'] = drought['FIPSYearMonth'].str.zfill(11)


,Date,FIPS,County,State,Sq_Mi_No_Drought,Sq_Mi_Abnormally_Dry,Sq_Mi_Moderate_Drought,Sq_Mi_Severe_Drought,Sq_Mi_Extreme_Drought,Sq_Mi_Exceptional_Drought,FIPSYearMonth,total_Sq_Mi,in_drought,Year
0,2017-12-26,5007,Benton County,AR,0.00,222.84,651.13,0.00,0.0,0.0,05007201712,873.97,1,2017
1,2017-12-19,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,05007201712,873.97,1,2017
2,2017-12-12,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,05007201712,873.97,1,2017
3,2017-12-05,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,05007201712,873.97,1,2017
4,2017-11-28,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,05007201711,873.97,1,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317048,2002-01-29,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002
317049,2002-01-22,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002
317050,2002-01-15,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002
317051,2002-01-08,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002


In [140]:
#crossing our fingers that we don't make any nans
merger1 = drought.merge(weather, on='FIPSYearMonth')
merger1

,Date,FIPS,County,State,Sq_Mi_No_Drought,Sq_Mi_Abnormally_Dry,Sq_Mi_Moderate_Drought,Sq_Mi_Severe_Drought,Sq_Mi_Extreme_Drought,Sq_Mi_Exceptional_Drought,FIPSYearMonth,total_Sq_Mi,in_drought,Year,Month,precip,mintemp,maxtemp,avgtemp,FIPSYear
0,2017-12-26,5007,Benton County,AR,0.00,222.84,651.13,0.00,0.0,0.0,05007201712,873.97,1,2017,12,1.92,28.8,48.9,38.9,050072017
1,2017-12-19,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,05007201712,873.97,1,2017,12,1.92,28.8,48.9,38.9,050072017
2,2017-12-12,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,05007201712,873.97,1,2017,12,1.92,28.8,48.9,38.9,050072017
3,2017-12-05,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,05007201712,873.97,1,2017,12,1.92,28.8,48.9,38.9,050072017
4,2017-11-28,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,05007201711,873.97,1,2017,11,1.58,40.7,62.7,51.7,050072017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307225,2002-01-29,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002,1,3.90,36.6,45.6,41.1,530292002
307226,2002-01-22,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002,1,3.90,36.6,45.6,41.1,530292002
307227,2002-01-15,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002,1,3.90,36.6,45.6,41.1,530292002
307228,2002-01-08,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002,1,3.90,36.6,45.6,41.1,530292002


In [141]:
#checking for nans and datatypes
merger1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307230 entries, 0 to 307229
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Date                       307230 non-null  object 
 1   FIPS                       307230 non-null  int64  
 2   County                     307230 non-null  object 
 3   State                      307230 non-null  object 
 4   Sq_Mi_No_Drought           307230 non-null  float64
 5   Sq_Mi_Abnormally_Dry       307230 non-null  float64
 6   Sq_Mi_Moderate_Drought     307230 non-null  float64
 7   Sq_Mi_Severe_Drought       307230 non-null  float64
 8   Sq_Mi_Extreme_Drought      307230 non-null  float64
 9   Sq_Mi_Exceptional_Drought  307230 non-null  float64
 10  FIPSYearMonth              307230 non-null  object 
 11  total_Sq_Mi                307230 non-null  float64
 12  in_drought                 307230 non-null  int32  
 13  Year                       30

The first merger went well! From the readout of the dataframe' head and tail we can see that Benton and Island county are correct and that the FIPS, Year, Month, and their respective combinations all match. We have duplicated values for the weather data, but that is by design--we need to forward fill the temperatures for each week in a given month. 

We will relabel the precipitation column as monthly_precip so we know don't mistake it for weekly precipitation durring an aggregation or later analysis.

Finally, let's check the USDA dataframe, 'coa' and make sure it is ready to be merged with our new dataframe, merger1.

In [142]:
coa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6064 entries, 0 to 6063
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Program               6064 non-null   object 
 1   Year                  6064 non-null   int64  
 2   State                 6064 non-null   object 
 3   County                6064 non-null   object 
 4   FIPS                  6064 non-null   int64  
 5   FIPSYear              6064 non-null   int64  
 6   Chem_expense_dollars  6064 non-null   float64
 7   Chem_operations       6064 non-null   float64
 8   Acres_operated        6064 non-null   float64
 9   Operations            6064 non-null   int64  
 10  Fert_expense_dollars  6064 non-null   float64
 11  Fert_operations       6064 non-null   float64
dtypes: float64(5), int64(4), object(3)
memory usage: 568.6+ KB


This time around we are merging on the FIPSYear column. This will need to be changed to a string and we will need to add leading zeros to it so it will match the FIPSYear column on merger1. We will keep the duplicated columns for now to allow us to verify the merger was correct. 

After verifying that the merger is right we are able to drop the extra Year, State, County, and FIPS columns from the dataframe 

In [143]:
#changing the FIPSYear column on coa to a string and filling in leading zeros
coa['FIPSYear'] = coa['FIPSYear'].astype(str)
coa['FIPSYear'] = coa['FIPSYear'].str.zfill(9)
coa

,Program,Year,State,County,FIPS,FIPSYear,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
0,CENSUS,2017,ARKANSAS,CONWAY,5029,050292017,1395000.0,375.0,171893.0,768,2350000.0,405.0
1,CENSUS,2017,ARKANSAS,FAULKNER,5045,050452017,921000.0,388.0,200539.0,1191,1559000.0,419.0
2,CENSUS,2017,ARKANSAS,GARLAND,5051,050512017,86000.0,104.0,33950.0,357,214000.0,98.0
3,CENSUS,2017,ARKANSAS,GRANT,5053,050532017,91000.0,102.0,49218.0,281,191000.0,118.0
4,CENSUS,2017,ARKANSAS,HOT SPRING,5059,050592017,167000.0,186.0,77331.0,563,243000.0,181.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6059,CENSUS,2002,WISCONSIN,MONROE,55081,550812002,2396000.0,817.0,351775.0,1938,3979000.0,1037.0
6060,CENSUS,2002,WISCONSIN,PEPIN,55091,550912002,1188000.0,298.0,111313.0,501,1478000.0,310.0
6061,CENSUS,2002,WISCONSIN,PIERCE,55093,550932002,2368000.0,659.0,267311.0,1510,4014000.0,691.0
6062,CENSUS,2002,WISCONSIN,ST CROIX,55109,551092002,2931000.0,701.0,310178.0,1864,4506000.0,773.0


In [144]:
#testing the merge
merger1.merge(coa, on='FIPSYear')

,Date,FIPS_x,County_x,State_x,Sq_Mi_No_Drought,Sq_Mi_Abnormally_Dry,Sq_Mi_Moderate_Drought,Sq_Mi_Severe_Drought,Sq_Mi_Extreme_Drought,Sq_Mi_Exceptional_Drought,FIPSYearMonth,total_Sq_Mi,in_drought,Year_x,Month,precip,mintemp,maxtemp,avgtemp,FIPSYear,Program,Year_y,State_y,County_y,FIPS_y,Chem_expense_dollars,Chem_operations,Acres_operated,Operations,Fert_expense_dollars,Fert_operations
0,2017-12-26,5007,Benton County,AR,0.00,222.84,651.13,0.00,0.0,0.0,05007201712,873.97,1,2017,12,1.92,28.8,48.9,38.9,050072017,CENSUS,2017,ARKANSAS,BENTON,5007,1072000.0,702.0,243753.0,1936,2967000.0,798.0
1,2017-12-19,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,05007201712,873.97,1,2017,12,1.92,28.8,48.9,38.9,050072017,CENSUS,2017,ARKANSAS,BENTON,5007,1072000.0,702.0,243753.0,1936,2967000.0,798.0
2,2017-12-12,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,05007201712,873.97,1,2017,12,1.92,28.8,48.9,38.9,050072017,CENSUS,2017,ARKANSAS,BENTON,5007,1072000.0,702.0,243753.0,1936,2967000.0,798.0
3,2017-12-05,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,05007201712,873.97,1,2017,12,1.92,28.8,48.9,38.9,050072017,CENSUS,2017,ARKANSAS,BENTON,5007,1072000.0,702.0,243753.0,1936,2967000.0,798.0
4,2017-11-28,5007,Benton County,AR,0.00,0.00,790.51,83.46,0.0,0.0,05007201711,873.97,1,2017,11,1.58,40.7,62.7,51.7,050072017,CENSUS,2017,ARKANSAS,BENTON,5007,1072000.0,702.0,243753.0,1936,2967000.0,798.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307016,2002-01-29,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002,1,3.90,36.6,45.6,41.1,530292002,CENSUS,2002,WASHINGTON,ISLAND,53029,85000.0,70.0,15018.0,348,135000.0,72.0
307017,2002-01-22,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002,1,3.90,36.6,45.6,41.1,530292002,CENSUS,2002,WASHINGTON,ISLAND,53029,85000.0,70.0,15018.0,348,135000.0,72.0
307018,2002-01-15,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002,1,3.90,36.6,45.6,41.1,530292002,CENSUS,2002,WASHINGTON,ISLAND,53029,85000.0,70.0,15018.0,348,135000.0,72.0
307019,2002-01-08,53029,Island County,WA,213.97,0.00,0.00,0.00,0.0,0.0,53029200201,213.97,0,2002,1,3.90,36.6,45.6,41.1,530292002,CENSUS,2002,WASHINGTON,ISLAND,53029,85000.0,70.0,15018.0,348,135000.0,72.0


In [145]:
#test looked good, lets make our final dataframe, soil
soil = merger1.merge(coa, on='FIPSYear')

In [146]:
soil.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307021 entries, 0 to 307020
Data columns (total 31 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Date                       307021 non-null  object 
 1   FIPS_x                     307021 non-null  int64  
 2   County_x                   307021 non-null  object 
 3   State_x                    307021 non-null  object 
 4   Sq_Mi_No_Drought           307021 non-null  float64
 5   Sq_Mi_Abnormally_Dry       307021 non-null  float64
 6   Sq_Mi_Moderate_Drought     307021 non-null  float64
 7   Sq_Mi_Severe_Drought       307021 non-null  float64
 8   Sq_Mi_Extreme_Drought      307021 non-null  float64
 9   Sq_Mi_Exceptional_Drought  307021 non-null  float64
 10  FIPSYearMonth              307021 non-null  object 
 11  total_Sq_Mi                307021 non-null  float64
 12  in_drought                 307021 non-null  int32  
 13  Year_x                     30

In [147]:
#writing the full soil dataframe to a CSV 
#soil.to_csv('soilRough.csv', index=False)

The weather and drought data is ok to combine as is. Precipitation is the only thing that would potentially need to be split up to fit the weekly drought data, but that is doable (albiet cumbersome).

You can make a subset for the 15 farmland states, but keep the cleaned full country data around. Might be interesting to analyse that a bit.

The data on organic operations might be useful for feature engineering

In [148]:
#coa.to_csv('coa', index=False)